In [1]:
!pip install aiofiles

DEPRECATION: colab 1.13.5 has a non-standard dependency specifier pytz>=2011n. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of colab or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [40]:
import os
import pandas as pd
import httpx
import asyncio
import aiofiles
from bs4 import BeautifulSoup
import requests
import re
import nest_asyncio
import time
import random

nest_asyncio.apply()

semaphore = asyncio.Semaphore(100)

pd.set_option('display.width', 100)

In [3]:
def create_page_url_with_genres_dict():
    webtoons_comic_page_urls = []
    webtoons_comic_genre = []

    originals_url = "https://www.webtoons.com/en/originals"
    r = requests.get(originals_url)
    soup = BeautifulSoup(r.content, 'html.parser')

    completed_comics = soup.find('div', class_='daily_section on').find_all('a', href=True)

    webtoons_comic_page_urls += [a['href'] for a in completed_comics]
    webtoons_comic_genre += [comic.find('p').text for comic in completed_comics]

    ongoing_comics_days = soup.find_all('div', class_=re.compile('daily_section\s+_list_\w*'))
    for ongoing_comic_day in ongoing_comics_days:
        daily_ongoing_comics = ongoing_comic_day.find_all('li')

        webtoons_comic_page_urls += [li.find('a', href=True)['href'] for li in daily_ongoing_comics]
        webtoons_comic_genre += [comic.find('p').text for comic in daily_ongoing_comics]

    return dict(zip(webtoons_comic_page_urls, webtoons_comic_genre))

def create_episode_url_with_genres_dict(page_url_genre_dict):
    episode_url_genre_dict = {}
    # Scrape episode URLs for each comic page URL
    for comic_page_url, genre in page_url_genre_dict.items():
        r = requests.get(comic_page_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for li in soup.find_all('li', class_='_episodeItem'):
            episode_url = li.find('a', href=True)['href']
            episode_url_genre_dict[episode_url] = genre
    return episode_url_genre_dict

page_url_genre_dict = create_page_url_with_genres_dict()
episode_url_genre_dict = create_episode_url_with_genres_dict(page_url_genre_dict)

In [41]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[0:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

finished https://www.webtoons.com/en/romance/marry-my-husband/episode-7/viewer?title_no=4572&episode_no=7 1 1.0404579639434814
finished https://www.webtoons.com/en/supernatural/mystical/episode-5/viewer?title_no=2128&episode_no=6 2 1.4135761260986328
finished https://www.webtoons.com/en/thriller/sweethome/ep-9/viewer?title_no=1285&episode_no=10 3 2.0862159729003906
finished https://www.webtoons.com/en/action/the-god-of-high-school/ep-32/viewer?title_no=66&episode_no=32 4 2.7301089763641357
finished https://www.webtoons.com/en/action/weakhero/ep-3/viewer?title_no=1726&episode_no=3 5 3.1044669151306152
finished https://www.webtoons.com/en/action/red-hood-outlaws/ep-3-the-lost-idol/viewer?title_no=4603&episode_no=3 6 3.6623239517211914
finished https://www.webtoons.com/en/romance/my-dear-cold-blooded-king/ep-21/viewer?title_no=961&episode_no=23 7 4.133944988250732
finished https://www.webtoons.com/en/action/noblesse/ep-49/viewer?title_no=87&episode_no=49 8 4.818147897720337
finished https

finished https://www.webtoons.com/en/fantasy/empyrea/s3-episode-68/viewer?title_no=1407&episode_no=74 65 37.37379598617554
finished https://www.webtoons.com/en/fantasy/21st-century-knights/s2-episode-71/viewer?title_no=2718&episode_no=71 66 37.791669845581055
finished https://www.webtoons.com/en/heartwarming/tata-the-cat/episode-39/viewer?title_no=5547&episode_no=39 67 38.25319004058838
finished https://www.webtoons.com/en/drama/knife-in-her-heart/episode-7/viewer?title_no=5043&episode_no=7 68 38.706092834472656
finished https://www.webtoons.com/en/romance/your-smile-is-a-trap/episode-131/viewer?title_no=2052&episode_no=131 69 39.21784210205078
finished https://www.webtoons.com/en/romance/not-your-typical-reincarnation-story/episode-30/viewer?title_no=5556&episode_no=30 70 39.88378882408142
finished https://www.webtoons.com/en/fantasy/a-martial-master-wakes-up-as-a-concubine/ep-26-candid-truths-behind-candied-fruits/viewer?title_no=5732&episode_no=26 71 40.48605680465698
finished https

In [42]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[1:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

finished https://www.webtoons.com/en/romance/marry-my-husband/episode-6/viewer?title_no=4572&episode_no=6 1 1.0603833198547363
finished https://www.webtoons.com/en/supernatural/mystical/episode-4/viewer?title_no=2128&episode_no=5 2 1.592621088027954
finished https://www.webtoons.com/en/thriller/sweethome/ep-8/viewer?title_no=1285&episode_no=9 3 2.0489389896392822
finished https://www.webtoons.com/en/action/the-god-of-high-school/ep-31/viewer?title_no=66&episode_no=31 4 2.8319590091705322
finished https://www.webtoons.com/en/action/weakhero/ep-2/viewer?title_no=1726&episode_no=2 5 3.552840232849121
finished https://www.webtoons.com/en/action/red-hood-outlaws/ep-2-dead-before-proven-alive/viewer?title_no=4603&episode_no=2 6 3.937181234359741
finished https://www.webtoons.com/en/romance/my-dear-cold-blooded-king/ep-20/viewer?title_no=961&episode_no=22 7 5.231391191482544
finished https://www.webtoons.com/en/action/noblesse/ep-48/viewer?title_no=87&episode_no=48 8 5.685591220855713
finishe

finished https://www.webtoons.com/en/fantasy/empyrea/s3-episode-67/viewer?title_no=1407&episode_no=73 65 56.221912145614624
finished https://www.webtoons.com/en/fantasy/21st-century-knights/s2-episode-70/viewer?title_no=2718&episode_no=70 66 56.81083106994629
finished https://www.webtoons.com/en/heartwarming/tata-the-cat/episode-38/viewer?title_no=5547&episode_no=38 67 57.98370814323425
finished https://www.webtoons.com/en/thriller/dead-queen/episode-17/viewer?title_no=5721&episode_no=17 68 59.58843421936035
finished https://www.webtoons.com/en/romance/your-smile-is-a-trap/episode-130/viewer?title_no=2052&episode_no=130 69 60.72760224342346
finished https://www.webtoons.com/en/romance/not-your-typical-reincarnation-story/episode-29/viewer?title_no=5556&episode_no=29 70 61.51933526992798
finished https://www.webtoons.com/en/fantasy/a-martial-master-wakes-up-as-a-concubine/ep-25-won-gae-or-another-im-gonna-find-ya/viewer?title_no=5732&episode_no=25 71 62.127745151519775
finished https://

In [43]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[2:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

finished https://www.webtoons.com/en/romance/marry-my-husband/episode-5/viewer?title_no=4572&episode_no=5 1 0.6068828105926514
finished https://www.webtoons.com/en/supernatural/mystical/episode-3/viewer?title_no=2128&episode_no=4 2 1.2176117897033691
finished https://www.webtoons.com/en/thriller/sweethome/ep-7/viewer?title_no=1285&episode_no=8 3 1.895265817642212
finished https://www.webtoons.com/en/action/the-god-of-high-school/ep-30/viewer?title_no=66&episode_no=30 4 3.706865072250366
finished https://www.webtoons.com/en/action/weakhero/ep-1/viewer?title_no=1726&episode_no=1 5 4.230475187301636
finished https://www.webtoons.com/en/action/red-hood-outlaws/ep-1-two-birds-one-clone/viewer?title_no=4603&episode_no=1 6 4.60973596572876
finished https://www.webtoons.com/en/romance/my-dear-cold-blooded-king/ep-19/viewer?title_no=961&episode_no=21 7 5.091093063354492
finished https://www.webtoons.com/en/action/noblesse/ep-47/viewer?title_no=87&episode_no=47 8 5.5019850730896
finished https:/

finished https://www.webtoons.com/en/romance/love-cycle/episode-36/viewer?title_no=4845&episode_no=36 65 39.27761220932007
finished https://www.webtoons.com/en/fantasy/21st-century-knights/s2-episode-69/viewer?title_no=2718&episode_no=69 66 39.79264998435974
finished https://www.webtoons.com/en/heartwarming/tata-the-cat/episode-37/viewer?title_no=5547&episode_no=37 67 40.45862793922424
finished https://www.webtoons.com/en/thriller/dead-queen/episode-16/viewer?title_no=5721&episode_no=16 68 41.49776601791382
finished https://www.webtoons.com/en/romance/your-smile-is-a-trap/episode-129/viewer?title_no=2052&episode_no=129 69 42.19483184814453
finished https://www.webtoons.com/en/romance/not-your-typical-reincarnation-story/episode-28/viewer?title_no=5556&episode_no=28 70 43.08694100379944
finished https://www.webtoons.com/en/fantasy/a-martial-master-wakes-up-as-a-concubine/ep-24-and-then-there-were-two/viewer?title_no=5732&episode_no=24 71 43.610422134399414
finished https://www.webtoons.

In [44]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[3:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

finished https://www.webtoons.com/en/romance/marry-my-husband/episode-4/viewer?title_no=4572&episode_no=4 1 1.1852922439575195
finished https://www.webtoons.com/en/supernatural/mystical/episode-2/viewer?title_no=2128&episode_no=3 2 2.0607190132141113
finished https://www.webtoons.com/en/thriller/sweethome/ep-6/viewer?title_no=1285&episode_no=7 3 2.6877851486206055
finished https://www.webtoons.com/en/action/the-god-of-high-school/ep-29/viewer?title_no=66&episode_no=29 4 3.226820945739746
finished https://www.webtoons.com/en/fantasy/the-matchmaking-baby-princess/episode-7/viewer?title_no=3066&episode_no=7 5 3.9805541038513184
finished https://www.webtoons.com/en/thriller/my-deepest-secret/ep-11-his-perspective-part-3/viewer?title_no=1580&episode_no=11 6 4.519363164901733
finished https://www.webtoons.com/en/romance/my-dear-cold-blooded-king/ep-18/viewer?title_no=961&episode_no=20 7 5.05176305770874
finished https://www.webtoons.com/en/action/noblesse/ep-46/viewer?title_no=87&episode_no=

finished https://www.webtoons.com/en/romance/love-cycle/episode-35/viewer?title_no=4845&episode_no=35 65 36.612566232681274
finished https://www.webtoons.com/en/fantasy/21st-century-knights/s2-episode-68/viewer?title_no=2718&episode_no=68 66 37.06754803657532
finished https://www.webtoons.com/en/heartwarming/tata-the-cat/episode-36/viewer?title_no=5547&episode_no=36 67 37.488911867141724
finished https://www.webtoons.com/en/thriller/dead-queen/episode-15/viewer?title_no=5721&episode_no=15 68 37.8868191242218
finished https://www.webtoons.com/en/romance/your-smile-is-a-trap/episode-128/viewer?title_no=2052&episode_no=128 69 38.32821989059448
finished https://www.webtoons.com/en/romance/adopted-by-a-murderous-duke-family/episode-6/viewer?title_no=6146&episode_no=6 70 38.70466423034668
finished https://www.webtoons.com/en/fantasy/a-martial-master-wakes-up-as-a-concubine/ep-23-attendants-please-this-is-soyeo-cheon-speaking/viewer?title_no=5732&episode_no=23 71 39.195521116256714
finished h

In [45]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[4:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

finished https://www.webtoons.com/en/romance/marry-my-husband/episode-3/viewer?title_no=4572&episode_no=3 1 1.0242819786071777
finished https://www.webtoons.com/en/supernatural/mystical/episode-1/viewer?title_no=2128&episode_no=2 2 1.4863240718841553
finished https://www.webtoons.com/en/thriller/sweethome/ep-5/viewer?title_no=1285&episode_no=6 3 1.9516820907592773
finished https://www.webtoons.com/en/action/the-god-of-high-school/ep-28/viewer?title_no=66&episode_no=28 4 2.508362293243408
finished https://www.webtoons.com/en/fantasy/the-matchmaking-baby-princess/episode-6/viewer?title_no=3066&episode_no=6 5 3.0648531913757324
finished https://www.webtoons.com/en/thriller/my-deepest-secret/ep-10-his-perspective-part-2/viewer?title_no=1580&episode_no=10 6 4.2934410572052
finished https://www.webtoons.com/en/romance/my-dear-cold-blooded-king/ep-17/viewer?title_no=961&episode_no=19 7 4.877215147018433
finished https://www.webtoons.com/en/action/noblesse/ep-45/viewer?title_no=87&episode_no=4

finished https://www.webtoons.com/en/romance/love-cycle/episode-34/viewer?title_no=4845&episode_no=34 65 48.062538146972656
finished https://www.webtoons.com/en/fantasy/21st-century-knights/s2-episode-67/viewer?title_no=2718&episode_no=67 66 48.56387710571289
finished https://www.webtoons.com/en/heartwarming/tata-the-cat/episode-35/viewer?title_no=5547&episode_no=35 67 48.99446702003479
finished https://www.webtoons.com/en/thriller/dead-queen/episode-14/viewer?title_no=5721&episode_no=14 68 49.508050203323364
finished https://www.webtoons.com/en/romance/your-smile-is-a-trap/episode-127/viewer?title_no=2052&episode_no=127 69 49.99475908279419
finished https://www.webtoons.com/en/romance/adopted-by-a-murderous-duke-family/episode-5/viewer?title_no=6146&episode_no=5 70 50.534722089767456
finished https://www.webtoons.com/en/fantasy/a-martial-master-wakes-up-as-a-concubine/ep-22-the-emperor-and-the-tteok-boy/viewer?title_no=5732&episode_no=22 71 50.92877507209778
finished https://www.webto

ConnectTimeout: 

In [46]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[5:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

finished https://www.webtoons.com/en/romance/marry-my-husband/episode-2/viewer?title_no=4572&episode_no=2 1 0.628669023513794
finished https://www.webtoons.com/en/supernatural/mystical/episode-0-prologue/viewer?title_no=2128&episode_no=1 2 1.0457959175109863
finished https://www.webtoons.com/en/romance/perfect-marriage-revenge/episode-7/viewer?title_no=3484&episode_no=7 3 1.6338520050048828
finished https://www.webtoons.com/en/action/the-god-of-high-school/ep-27/viewer?title_no=66&episode_no=27 4 2.0546069145202637
finished https://www.webtoons.com/en/fantasy/the-matchmaking-baby-princess/episode-5/viewer?title_no=3066&episode_no=5 5 2.610534906387329
finished https://www.webtoons.com/en/thriller/my-deepest-secret/ep-9-his-perspective-part-1/viewer?title_no=1580&episode_no=9 6 3.114706039428711
finished https://www.webtoons.com/en/romance/the-guy-with-pretty-lips/episode-10/viewer?title_no=5435&episode_no=10 7 3.5551388263702393
finished https://www.webtoons.com/en/action/noblesse/ep-4

finished https://www.webtoons.com/en/fantasy/lord-of-goblins/episode-51/viewer?title_no=5227&episode_no=52 64 35.54662799835205
finished https://www.webtoons.com/en/romance/love-cycle/episode-33/viewer?title_no=4845&episode_no=33 65 35.961302042007446
finished https://www.webtoons.com/en/fantasy/21st-century-knights/s2-episode-66/viewer?title_no=2718&episode_no=66 66 36.51865291595459
finished https://www.webtoons.com/en/heartwarming/tata-the-cat/episode-34/viewer?title_no=5547&episode_no=34 67 36.89182114601135
finished https://www.webtoons.com/en/thriller/dead-queen/episode-13/viewer?title_no=5721&episode_no=13 68 37.280832052230835
finished https://www.webtoons.com/en/romance/your-smile-is-a-trap/episode-126/viewer?title_no=2052&episode_no=126 69 37.719578981399536
finished https://www.webtoons.com/en/romance/adopted-by-a-murderous-duke-family/episode-4/viewer?title_no=6146&episode_no=4 70 38.07817101478577
finished https://www.webtoons.com/en/fantasy/a-martial-master-wakes-up-as-a-

ReadTimeout: 

In [47]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[6:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

finished https://www.webtoons.com/en/romance/marry-my-husband/episode-1/viewer?title_no=4572&episode_no=1 1 0.6656651496887207
finished https://www.webtoons.com/en/romance/the-male-leads-girl-friend/episode-7/viewer?title_no=3412&episode_no=7 2 1.0615291595458984
finished https://www.webtoons.com/en/romance/perfect-marriage-revenge/episode-6/viewer?title_no=3484&episode_no=6 3 1.4419100284576416
finished https://www.webtoons.com/en/action/the-god-of-high-school/ep-26/viewer?title_no=66&episode_no=26 4 1.8753001689910889
finished https://www.webtoons.com/en/fantasy/the-matchmaking-baby-princess/episode-4/viewer?title_no=3066&episode_no=4 5 2.94423508644104
finished https://www.webtoons.com/en/thriller/my-deepest-secret/ep-8-my-troubles-part-3/viewer?title_no=1580&episode_no=8 6 4.666685104370117
finished https://www.webtoons.com/en/romance/the-guy-with-pretty-lips/episode-9/viewer?title_no=5435&episode_no=9 7 5.193883895874023
finished https://www.webtoons.com/en/action/noblesse/ep-43/v

finished https://www.webtoons.com/en/fantasy/lord-of-goblins/episode-50/viewer?title_no=5227&episode_no=51 64 33.40638327598572
finished https://www.webtoons.com/en/romance/love-cycle/episode-32/viewer?title_no=4845&episode_no=32 65 34.063329219818115
finished https://www.webtoons.com/en/fantasy/21st-century-knights/s2-episode-65/viewer?title_no=2718&episode_no=65 66 34.67008113861084
finished https://www.webtoons.com/en/heartwarming/tata-the-cat/episode-33/viewer?title_no=5547&episode_no=33 67 35.10578894615173
finished https://www.webtoons.com/en/thriller/dead-queen/episode-12/viewer?title_no=5721&episode_no=12 68 35.5615119934082
finished https://www.webtoons.com/en/romance/your-smile-is-a-trap/episode-125/viewer?title_no=2052&episode_no=125 69 36.146673917770386
finished https://www.webtoons.com/en/romance/adopted-by-a-murderous-duke-family/episode-3/viewer?title_no=6146&episode_no=3 70 36.516109228134155
finished https://www.webtoons.com/en/fantasy/a-martial-master-wakes-up-as-a-c

ConnectTimeout: 

In [48]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[7:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

finished https://www.webtoons.com/en/drama/my-id-is-gangnam-beauty/ep-9-reunion/viewer?title_no=3064&episode_no=9 1 1.076956033706665
finished https://www.webtoons.com/en/romance/the-male-leads-girl-friend/episode-6/viewer?title_no=3412&episode_no=6 2 1.6303799152374268
finished https://www.webtoons.com/en/romance/perfect-marriage-revenge/episode-5/viewer?title_no=3484&episode_no=5 3 2.190272808074951
finished https://www.webtoons.com/en/action/the-god-of-high-school/ep-25/viewer?title_no=66&episode_no=25 4 2.610318899154663
finished https://www.webtoons.com/en/fantasy/the-matchmaking-baby-princess/episode-3/viewer?title_no=3066&episode_no=3 5 4.199665069580078
finished https://www.webtoons.com/en/thriller/my-deepest-secret/ep-7-my-troubles-part-2/viewer?title_no=1580&episode_no=7 6 4.606826066970825
finished https://www.webtoons.com/en/romance/the-guy-with-pretty-lips/episode-8/viewer?title_no=5435&episode_no=8 7 5.677819013595581
finished https://www.webtoons.com/en/action/noblesse/e

finished https://www.webtoons.com/en/fantasy/lord-of-goblins/episode-49/viewer?title_no=5227&episode_no=50 64 36.94325804710388
finished https://www.webtoons.com/en/romance/love-cycle/episode-31/viewer?title_no=4845&episode_no=31 65 37.339215993881226
finished https://www.webtoons.com/en/fantasy/21st-century-knights/s2-episode-64/viewer?title_no=2718&episode_no=64 66 37.84736490249634
finished https://www.webtoons.com/en/heartwarming/tata-the-cat/episode-32/viewer?title_no=5547&episode_no=32 67 38.531749963760376
finished https://www.webtoons.com/en/thriller/dead-queen/episode-11/viewer?title_no=5721&episode_no=11 68 39.11589312553406
finished https://www.webtoons.com/en/romance/your-smile-is-a-trap/episode-124/viewer?title_no=2052&episode_no=124 69 39.64421486854553
finished https://www.webtoons.com/en/romance/adopted-by-a-murderous-duke-family/episode-2/viewer?title_no=6146&episode_no=2 70 40.09110403060913
finished https://www.webtoons.com/en/fantasy/a-martial-master-wakes-up-as-a-c

ConnectTimeout: 

In [49]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[8:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

finished https://www.webtoons.com/en/drama/my-id-is-gangnam-beauty/ep-8-an-unexpected-misunderstanding/viewer?title_no=3064&episode_no=8 1 3.198167085647583
finished https://www.webtoons.com/en/romance/the-male-leads-girl-friend/episode-5/viewer?title_no=3412&episode_no=5 2 3.9283480644226074
finished https://www.webtoons.com/en/romance/perfect-marriage-revenge/episode-4/viewer?title_no=3484&episode_no=4 3 4.522608280181885
finished https://www.webtoons.com/en/romance/a-heart-for-the-emperor/episode-3/viewer?title_no=4269&episode_no=3 4 5.513520956039429
finished https://www.webtoons.com/en/fantasy/the-matchmaking-baby-princess/episode-2/viewer?title_no=3066&episode_no=2 5 6.019787073135376
finished https://www.webtoons.com/en/thriller/my-deepest-secret/ep-6-my-troubles-part-1/viewer?title_no=1580&episode_no=6 6 6.612442255020142
finished https://www.webtoons.com/en/romance/the-guy-with-pretty-lips/episode-7/viewer?title_no=5435&episode_no=7 7 7.039803981781006
finished https://www.web

finished https://www.webtoons.com/en/action/the-primal-hunter/episode-19/viewer?title_no=5970&episode_no=19 63 40.468122243881226
finished https://www.webtoons.com/en/fantasy/lord-of-goblins/episode-48/viewer?title_no=5227&episode_no=49 64 40.978700160980225
finished https://www.webtoons.com/en/romance/love-cycle/episode-30/viewer?title_no=4845&episode_no=30 65 41.38773822784424
finished https://www.webtoons.com/en/fantasy/21st-century-knights/s2-episode-63/viewer?title_no=2718&episode_no=63 66 41.80765390396118
finished https://www.webtoons.com/en/heartwarming/tata-the-cat/episode-31/viewer?title_no=5547&episode_no=31 67 42.30366587638855
finished https://www.webtoons.com/en/thriller/dead-queen/episode-10/viewer?title_no=5721&episode_no=10 68 42.68043899536133
finished https://www.webtoons.com/en/romance/your-smile-is-a-trap/episode-123/viewer?title_no=2052&episode_no=123 69 43.235909938812256
finished https://www.webtoons.com/en/romance/adopted-by-a-murderous-duke-family/episode-1/vi

ConnectTimeout: 

In [50]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[9:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

finished https://www.webtoons.com/en/drama/my-id-is-gangnam-beauty/ep-7-group-project/viewer?title_no=3064&episode_no=7 1 2.2852909564971924
finished https://www.webtoons.com/en/romance/the-male-leads-girl-friend/episode-4/viewer?title_no=3412&episode_no=4 2 2.737466812133789
finished https://www.webtoons.com/en/romance/perfect-marriage-revenge/episode-3/viewer?title_no=3484&episode_no=3 3 3.566243886947632
finished https://www.webtoons.com/en/romance/a-heart-for-the-emperor/episode-2/viewer?title_no=4269&episode_no=2 4 4.08781886100769
finished https://www.webtoons.com/en/fantasy/the-matchmaking-baby-princess/episode-1/viewer?title_no=3066&episode_no=1 5 4.450257778167725
finished https://www.webtoons.com/en/thriller/my-deepest-secret/ep-5-my-first-date-part-2/viewer?title_no=1580&episode_no=5 6 4.874242067337036
finished https://www.webtoons.com/en/romance/the-guy-with-pretty-lips/episode-6/viewer?title_no=5435&episode_no=6 7 5.61532998085022
finished https://www.webtoons.com/en/dram

finished https://www.webtoons.com/en/action/the-primal-hunter/episode-18/viewer?title_no=5970&episode_no=18 63 39.508283853530884
finished https://www.webtoons.com/en/fantasy/lord-of-goblins/episode-47/viewer?title_no=5227&episode_no=48 64 40.031975984573364
finished https://www.webtoons.com/en/romance/love-cycle/episode-29/viewer?title_no=4845&episode_no=29 65 41.003323793411255
finished https://www.webtoons.com/en/action/black-knight/episode-52/viewer?title_no=5314&episode_no=52 66 41.96997594833374
finished https://www.webtoons.com/en/drama/denounce/ep-65-take-everything-down-to-their-socks/viewer?title_no=5124&episode_no=65 67 42.90223288536072
finished https://www.webtoons.com/en/thriller/dead-queen/episode-9/viewer?title_no=5721&episode_no=9 68 43.33110594749451
finished https://www.webtoons.com/en/supernatural/time-and-time-again/s2-ep-70-new-normal-mid-season-finale/viewer?title_no=3758&episode_no=70 69 44.1362030506134
finished https://www.webtoons.com/en/action/omniscient-rea

ReadTimeout: 

In [51]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[10:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

finished https://www.webtoons.com/en/drama/my-id-is-gangnam-beauty/ep-6-why-are-you-doing-this-to-me/viewer?title_no=3064&episode_no=6 1 0.9878089427947998
finished https://www.webtoons.com/en/romance/the-male-leads-girl-friend/episode-3/viewer?title_no=3412&episode_no=3 2 1.4449899196624756
finished https://www.webtoons.com/en/romance/perfect-marriage-revenge/episode-2/viewer?title_no=3484&episode_no=2 3 1.9870038032531738
finished https://www.webtoons.com/en/romance/a-heart-for-the-emperor/episode-1/viewer?title_no=4269&episode_no=1 4 2.4684290885925293
finished https://www.webtoons.com/en/romance/to-my-first-love/episode-4/viewer?title_no=5064&episode_no=4 5 2.9768409729003906
finished https://www.webtoons.com/en/thriller/my-deepest-secret/ep-4-my-first-date-part-1/viewer?title_no=1580&episode_no=4 6 3.405120849609375
finished https://www.webtoons.com/en/romance/the-guy-with-pretty-lips/episode-5/viewer?title_no=5435&episode_no=5 7 3.8938980102539062
finished https://www.webtoons.co

finished https://www.webtoons.com/en/fantasy/tower-of-god/season-3-ep-189/viewer?title_no=95&episode_no=607 62 33.881654024124146
finished https://www.webtoons.com/en/action/the-primal-hunter/episode-17/viewer?title_no=5970&episode_no=17 63 34.281188011169434
finished https://www.webtoons.com/en/fantasy/lord-of-goblins/episode-46/viewer?title_no=5227&episode_no=47 64 34.73769283294678
finished https://www.webtoons.com/en/romance/love-cycle/episode-28/viewer?title_no=4845&episode_no=28 65 35.33564591407776
finished https://www.webtoons.com/en/action/black-knight/episode-51/viewer?title_no=5314&episode_no=51 66 35.770978927612305
finished https://www.webtoons.com/en/drama/denounce/ep-64-originally-ours/viewer?title_no=5124&episode_no=64 67 36.37406301498413
finished https://www.webtoons.com/en/thriller/dead-queen/episode-8/viewer?title_no=5721&episode_no=8 68 36.863924980163574
finished https://www.webtoons.com/en/supernatural/time-and-time-again/s2-ep-69-natural-order/viewer?title_no=37

ConnectTimeout: 

In [52]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[11:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

finished https://www.webtoons.com/en/drama/my-id-is-gangnam-beauty/ep-5-friends/viewer?title_no=3064&episode_no=5 1 2.366798162460327
finished https://www.webtoons.com/en/romance/the-male-leads-girl-friend/episode-2/viewer?title_no=3412&episode_no=2 2 3.274730920791626
finished https://www.webtoons.com/en/romance/perfect-marriage-revenge/episode-1/viewer?title_no=3484&episode_no=1 3 3.7851309776306152
finished https://www.webtoons.com/en/romance/hashtag-muted/episode-7/viewer?title_no=5798&episode_no=7 4 4.355793237686157
finished https://www.webtoons.com/en/romance/to-my-first-love/episode-3/viewer?title_no=5064&episode_no=3 5 4.842532157897949
finished https://www.webtoons.com/en/thriller/my-deepest-secret/ep-3-my-worries/viewer?title_no=1580&episode_no=3 6 5.2867021560668945
finished https://www.webtoons.com/en/romance/the-guy-with-pretty-lips/episode-4/viewer?title_no=5435&episode_no=4 7 5.740377187728882
finished https://www.webtoons.com/en/romance/see-you-in-my-19th-life/episode-

finished https://www.webtoons.com/en/action/the-primal-hunter/episode-16/viewer?title_no=5970&episode_no=16 63 38.39072918891907
finished https://www.webtoons.com/en/fantasy/lord-of-goblins/episode-45/viewer?title_no=5227&episode_no=46 64 38.9034161567688
finished https://www.webtoons.com/en/romance/love-cycle/episode-27/viewer?title_no=4845&episode_no=27 65 39.388094902038574
finished https://www.webtoons.com/en/action/black-knight/episode-50/viewer?title_no=5314&episode_no=50 66 39.87034010887146
finished https://www.webtoons.com/en/drama/denounce/ep-63-a-trapped-rat/viewer?title_no=5124&episode_no=63 67 40.40777897834778
finished https://www.webtoons.com/en/drama/the-producers-second-life/episode-14/viewer?title_no=5046&episode_no=14 68 40.903199195861816
finished https://www.webtoons.com/en/supernatural/time-and-time-again/s2-ep-68-oasis/viewer?title_no=3758&episode_no=68 69 41.36326026916504
finished https://www.webtoons.com/en/action/omniscient-reader/episode-197/viewer?title_no=

ConnectTimeout: 

In [53]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[12:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

finished https://www.webtoons.com/en/drama/my-id-is-gangnam-beauty/ep-4-oops-my-bad/viewer?title_no=3064&episode_no=4 1 0.6379959583282471
finished https://www.webtoons.com/en/romance/the-male-leads-girl-friend/episode-1/viewer?title_no=3412&episode_no=1 2 1.0555808544158936
finished https://www.webtoons.com/en/romance/secretary-out-of-order/episode-7/viewer?title_no=3790&episode_no=7 3 1.4590380191802979
finished https://www.webtoons.com/en/romance/hashtag-muted/episode-6/viewer?title_no=5798&episode_no=6 4 1.857658863067627
finished https://www.webtoons.com/en/romance/to-my-first-love/episode-2/viewer?title_no=5064&episode_no=2 5 2.271146059036255
finished https://www.webtoons.com/en/thriller/my-deepest-secret/ep-2-my-senior/viewer?title_no=1580&episode_no=2 6 2.679626941680908
finished https://www.webtoons.com/en/romance/the-guy-with-pretty-lips/episode-3/viewer?title_no=5435&episode_no=3 7 3.837167978286743
finished https://www.webtoons.com/en/romance/see-you-in-my-19th-life/episod

finished https://www.webtoons.com/en/action/the-primal-hunter/episode-15/viewer?title_no=5970&episode_no=15 63 31.768476009368896
finished https://www.webtoons.com/en/romance/the-doctors-are-out/s3-episode-149/viewer?title_no=1910&episode_no=149 64 32.36532473564148
finished https://www.webtoons.com/en/drama/elegant-desire/episode-21/viewer?title_no=5836&episode_no=21 65 32.767597913742065
finished https://www.webtoons.com/en/action/black-knight/episode-49/viewer?title_no=5314&episode_no=49 66 33.17614483833313
finished https://www.webtoons.com/en/drama/denounce/ep-62-an-eye-for-an-eye/viewer?title_no=5124&episode_no=62 67 33.764748096466064
finished https://www.webtoons.com/en/drama/the-producers-second-life/episode-13/viewer?title_no=5046&episode_no=13 68 34.23350501060486
finished https://www.webtoons.com/en/supernatural/time-and-time-again/s2-ep-67-gentle-promise/viewer?title_no=3758&episode_no=67 69 34.77019190788269
finished https://www.webtoons.com/en/action/omniscient-reader/ep

ReadTimeout: 

In [54]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[13:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

finished https://www.webtoons.com/en/drama/my-id-is-gangnam-beauty/ep-3-a-mistake/viewer?title_no=3064&episode_no=3 1 1.6814541816711426
finished https://www.webtoons.com/en/thriller/bastard/ep-10/viewer?title_no=485&episode_no=11 2 2.3222010135650635
finished https://www.webtoons.com/en/romance/secretary-out-of-order/episode-6/viewer?title_no=3790&episode_no=6 3 2.7887191772460938
finished https://www.webtoons.com/en/romance/hashtag-muted/episode-5/viewer?title_no=5798&episode_no=5 4 3.2323861122131348
finished https://www.webtoons.com/en/romance/to-my-first-love/episode-1/viewer?title_no=5064&episode_no=1 5 5.8879899978637695
finished https://www.webtoons.com/en/drama/shes-hopeless/episode-7/viewer?title_no=2767&episode_no=7 6 6.445235967636108
finished https://www.webtoons.com/en/romance/the-guy-with-pretty-lips/episode-2/viewer?title_no=5435&episode_no=2 7 7.017873048782349
finished https://www.webtoons.com/en/romance/see-you-in-my-19th-life/episode-5/viewer?title_no=1266&episode_n

finished https://www.webtoons.com/en/action/the-primal-hunter/episode-14/viewer?title_no=5970&episode_no=14 63 45.331539154052734
finished https://www.webtoons.com/en/romance/the-doctors-are-out/s3-episode-148/viewer?title_no=1910&episode_no=148 64 45.91382598876953
finished https://www.webtoons.com/en/drama/elegant-desire/episode-20/viewer?title_no=5836&episode_no=20 65 46.460299015045166
finished https://www.webtoons.com/en/action/black-knight/episode-48/viewer?title_no=5314&episode_no=48 66 46.86462616920471
finished https://www.webtoons.com/en/drama/denounce/ep-61-spy/viewer?title_no=5124&episode_no=61 67 47.5463171005249
finished https://www.webtoons.com/en/drama/the-producers-second-life/episode-12/viewer?title_no=5046&episode_no=12 68 47.95206594467163
finished https://www.webtoons.com/en/supernatural/time-and-time-again/s2-ep-66-respite/viewer?title_no=3758&episode_no=66 69 48.59729814529419
finished https://www.webtoons.com/en/action/omniscient-reader/episode-195/viewer?title_

ConnectTimeout: 

In [55]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[14:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

finished https://www.webtoons.com/en/drama/my-id-is-gangnam-beauty/ep-2-i-was-born-pretty/viewer?title_no=3064&episode_no=2 1 1.6399738788604736
finished https://www.webtoons.com/en/thriller/bastard/ep-9/viewer?title_no=485&episode_no=10 2 3.5253119468688965
finished https://www.webtoons.com/en/romance/secretary-out-of-order/episode-5/viewer?title_no=3790&episode_no=5 3 4.053600788116455
finished https://www.webtoons.com/en/romance/hashtag-muted/episode-4/viewer?title_no=5798&episode_no=4 4 7.63300085067749
finished https://www.webtoons.com/en/romance/age-matters/episode-13/viewer?title_no=1364&episode_no=14 5 8.246922016143799
finished https://www.webtoons.com/en/drama/shes-hopeless/episode-6/viewer?title_no=2767&episode_no=6 6 8.650205850601196
finished https://www.webtoons.com/en/romance/the-guy-with-pretty-lips/episode-1/viewer?title_no=5435&episode_no=1 7 9.321486949920654
finished https://www.webtoons.com/en/romance/see-you-in-my-19th-life/episode-4/viewer?title_no=1266&episode_n

finished https://www.webtoons.com/en/action/the-primal-hunter/episode-13/viewer?title_no=5970&episode_no=13 63 48.9788498878479
finished https://www.webtoons.com/en/romance/the-doctors-are-out/s3-episode-147/viewer?title_no=1910&episode_no=147 64 49.67104172706604
finished https://www.webtoons.com/en/drama/elegant-desire/episode-19/viewer?title_no=5836&episode_no=19 65 50.138500928878784
finished https://www.webtoons.com/en/action/black-knight/episode-47/viewer?title_no=5314&episode_no=47 66 50.89582586288452
finished https://www.webtoons.com/en/drama/denounce/ep-60-trickery/viewer?title_no=5124&episode_no=60 67 51.44097089767456
finished https://www.webtoons.com/en/drama/the-producers-second-life/episode-11/viewer?title_no=5046&episode_no=11 68 52.15289282798767
finished https://www.webtoons.com/en/supernatural/time-and-time-again/s2-ep-65-easy-does-it/viewer?title_no=3758&episode_no=65 69 52.634055852890015
finished https://www.webtoons.com/en/action/omniscient-reader/episode-194/vie

ReadTimeout: 

In [57]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[15:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

finished https://www.webtoons.com/en/drama/my-id-is-gangnam-beauty/ep-1-im-pretty-from-now-on/viewer?title_no=3064&episode_no=1 1 0.9626328945159912
finished https://www.webtoons.com/en/thriller/bastard/ep-8/viewer?title_no=485&episode_no=9 2 1.7088158130645752
finished https://www.webtoons.com/en/romance/secretary-out-of-order/episode-4/viewer?title_no=3790&episode_no=4 3 2.8163459300994873
finished https://www.webtoons.com/en/romance/hashtag-muted/episode-3/viewer?title_no=5798&episode_no=3 4 3.735975980758667
finished https://www.webtoons.com/en/romance/age-matters/episode-12/viewer?title_no=1364&episode_no=13 5 5.277261734008789
finished https://www.webtoons.com/en/drama/shes-hopeless/episode-5/viewer?title_no=2767&episode_no=5 6 5.9920079708099365
finished https://www.webtoons.com/en/romance/there-must-be-happy-endings/episode-7/viewer?title_no=3065&episode_no=7 7 6.657211780548096
finished https://www.webtoons.com/en/romance/see-you-in-my-19th-life/episode-3/viewer?title_no=1266&

finished https://www.webtoons.com/en/romance/the-doctors-are-out/s3-episode-146/viewer?title_no=1910&episode_no=146 64 48.74343299865723
finished https://www.webtoons.com/en/drama/elegant-desire/episode-18/viewer?title_no=5836&episode_no=18 65 49.26902198791504
finished https://www.webtoons.com/en/action/black-knight/episode-46/viewer?title_no=5314&episode_no=46 66 50.482627868652344
finished https://www.webtoons.com/en/drama/denounce/ep-59-public-enemy/viewer?title_no=5124&episode_no=59 67 50.95763087272644
finished https://www.webtoons.com/en/drama/the-producers-second-life/episode-10/viewer?title_no=5046&episode_no=10 68 51.39022374153137
finished https://www.webtoons.com/en/supernatural/time-and-time-again/s2-ep-64-vested-interest/viewer?title_no=3758&episode_no=64 69 52.11119890213013
finished https://www.webtoons.com/en/action/omniscient-reader/episode-193/viewer?title_no=2154&episode_no=194 70 53.08842492103577
finished https://www.webtoons.com/en/comedy/not-so-shoujo-love-story

ConnectTimeout: 

In [58]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[16:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

finished https://www.webtoons.com/en/romance/the-dukes-cursed-charm/episode-3/viewer?title_no=4678&episode_no=3 1 2.187537908554077
finished https://www.webtoons.com/en/thriller/bastard/ep-7/viewer?title_no=485&episode_no=8 2 3.413862943649292
finished https://www.webtoons.com/en/romance/secretary-out-of-order/episode-3/viewer?title_no=3790&episode_no=3 3 5.241491079330444
finished https://www.webtoons.com/en/romance/hashtag-muted/episode-2/viewer?title_no=5798&episode_no=2 4 5.777928113937378
finished https://www.webtoons.com/en/romance/age-matters/episode-11/viewer?title_no=1364&episode_no=12 5 6.249267101287842
finished https://www.webtoons.com/en/drama/shes-hopeless/episode-4/viewer?title_no=2767&episode_no=4 6 8.187044143676758
finished https://www.webtoons.com/en/romance/there-must-be-happy-endings/episode-6/viewer?title_no=3065&episode_no=6 7 9.185811996459961
finished https://www.webtoons.com/en/romance/see-you-in-my-19th-life/episode-2/viewer?title_no=1266&episode_no=2 8 10.73

finished https://www.webtoons.com/en/romance/the-doctors-are-out/s3-episode-145/viewer?title_no=1910&episode_no=145 64 65.52713203430176
finished https://www.webtoons.com/en/drama/elegant-desire/episode-17/viewer?title_no=5836&episode_no=17 65 66.1760311126709
finished https://www.webtoons.com/en/action/black-knight/episode-45/viewer?title_no=5314&episode_no=45 66 67.48160910606384
finished https://www.webtoons.com/en/drama/denounce/ep-58-a-new-star-appears/viewer?title_no=5124&episode_no=58 67 68.18848419189453
finished https://www.webtoons.com/en/drama/the-producers-second-life/episode-9/viewer?title_no=5046&episode_no=9 68 69.08526611328125
finished https://www.webtoons.com/en/supernatural/time-and-time-again/s2-ep-63-employee-of-the-month/viewer?title_no=3758&episode_no=63 69 71.45829606056213
finished https://www.webtoons.com/en/action/omniscient-reader/episode-192/viewer?title_no=2154&episode_no=193 70 72.47295212745667
finished https://www.webtoons.com/en/comedy/not-so-shoujo-lo

ReadTimeout: 

In [59]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[17:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

finished https://www.webtoons.com/en/romance/the-dukes-cursed-charm/episode-2/viewer?title_no=4678&episode_no=2 1 2.472376823425293
finished https://www.webtoons.com/en/thriller/bastard/ep-6/viewer?title_no=485&episode_no=7 2 2.904709815979004
finished https://www.webtoons.com/en/romance/secretary-out-of-order/episode-2/viewer?title_no=3790&episode_no=2 3 3.4792840480804443
finished https://www.webtoons.com/en/romance/hashtag-muted/episode-1/viewer?title_no=5798&episode_no=1 4 3.9174580574035645
finished https://www.webtoons.com/en/romance/age-matters/episode-10/viewer?title_no=1364&episode_no=11 5 5.599654912948608
finished https://www.webtoons.com/en/drama/shes-hopeless/episode-3/viewer?title_no=2767&episode_no=3 6 6.180820941925049
finished https://www.webtoons.com/en/romance/there-must-be-happy-endings/episode-5/viewer?title_no=3065&episode_no=5 7 6.9441139698028564
finished https://www.webtoons.com/en/romance/see-you-in-my-19th-life/episode-1/viewer?title_no=1266&episode_no=1 8 7.

finished https://www.webtoons.com/en/romance/the-doctors-are-out/s3-episode-144/viewer?title_no=1910&episode_no=144 64 51.471582889556885
finished https://www.webtoons.com/en/drama/elegant-desire/episode-16/viewer?title_no=5836&episode_no=16 65 51.92262291908264
finished https://www.webtoons.com/en/action/black-knight/episode-44/viewer?title_no=5314&episode_no=44 66 52.82598900794983
finished https://www.webtoons.com/en/drama/denounce/ep-57-lets-fight/viewer?title_no=5124&episode_no=57 67 53.339706897735596
finished https://www.webtoons.com/en/drama/the-producers-second-life/episode-8/viewer?title_no=5046&episode_no=8 68 53.881980895996094
finished https://www.webtoons.com/en/supernatural/time-and-time-again/s2-ep-62-notions/viewer?title_no=3758&episode_no=62 69 54.34187078475952
finished https://www.webtoons.com/en/action/omniscient-reader/episode-191/viewer?title_no=2154&episode_no=192 70 54.982993841171265
finished https://www.webtoons.com/en/comedy/not-so-shoujo-love-story/s2-episo

ReadTimeout: 

In [60]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[18:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

finished https://www.webtoons.com/en/romance/the-dukes-cursed-charm/episode-1/viewer?title_no=4678&episode_no=1 1 0.8895800113677979
finished https://www.webtoons.com/en/thriller/bastard/ep-5/viewer?title_no=485&episode_no=6 2 1.5387990474700928
finished https://www.webtoons.com/en/romance/secretary-out-of-order/episode-1/viewer?title_no=3790&episode_no=1 3 2.3652148246765137
finished https://www.webtoons.com/en/thriller/its-mine/episode-7/viewer?title_no=2010&episode_no=8 4 3.4903268814086914
finished https://www.webtoons.com/en/romance/age-matters/episode-9/viewer?title_no=1364&episode_no=10 5 4.77462911605835
finished https://www.webtoons.com/en/drama/shes-hopeless/episode-2/viewer?title_no=2767&episode_no=2 6 6.051542043685913
finished https://www.webtoons.com/en/romance/there-must-be-happy-endings/episode-4/viewer?title_no=3065&episode_no=4 7 6.809530973434448
finished https://www.webtoons.com/en/action/a-useless-villain/episode-3/viewer?title_no=2969&episode_no=3 8 7.267100095748

finished https://www.webtoons.com/en/romance/the-doctors-are-out/s3-episode-143/viewer?title_no=1910&episode_no=143 64 47.221566915512085
finished https://www.webtoons.com/en/drama/elegant-desire/episode-15/viewer?title_no=5836&episode_no=15 65 48.259522914886475
finished https://www.webtoons.com/en/action/black-knight/episode-43/viewer?title_no=5314&episode_no=43 66 49.20316791534424
finished https://www.webtoons.com/en/drama/denounce/ep-56-miracle-of-a-blue-rose/viewer?title_no=5124&episode_no=56 67 49.71111583709717
finished https://www.webtoons.com/en/drama/the-producers-second-life/episode-7/viewer?title_no=5046&episode_no=7 68 50.38785982131958
finished https://www.webtoons.com/en/action/code-adam/s3-episode-164-season-3-finale/viewer?title_no=1657&episode_no=166 69 51.1665358543396
finished https://www.webtoons.com/en/action/omniscient-reader/episode-190/viewer?title_no=2154&episode_no=191 70 52.09930396080017
finished https://www.webtoons.com/en/drama/acception/episode-259/view

ConnectTimeout: 

In [61]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[19:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

finished https://www.webtoons.com/en/romance/my-younger-brothers-friend/episode-6/viewer?title_no=3163&episode_no=7 1 0.6438052654266357
finished https://www.webtoons.com/en/thriller/bastard/ep-4/viewer?title_no=485&episode_no=5 2 1.2467122077941895
finished https://www.webtoons.com/en/comedy/mage-and-demon-queen/episode-19/viewer?title_no=1438&episode_no=19 3 1.711266040802002
finished https://www.webtoons.com/en/thriller/its-mine/episode-6/viewer?title_no=2010&episode_no=7 4 2.189390182495117
finished https://www.webtoons.com/en/romance/age-matters/episode-8/viewer?title_no=1364&episode_no=9 5 2.75008225440979
finished https://www.webtoons.com/en/drama/shes-hopeless/episode-1/viewer?title_no=2767&episode_no=1 6 3.326343297958374
finished https://www.webtoons.com/en/romance/there-must-be-happy-endings/episode-3/viewer?title_no=3065&episode_no=3 7 3.916076183319092
finished https://www.webtoons.com/en/action/a-useless-villain/episode-2/viewer?title_no=2969&episode_no=2 8 5.102678060531

finished https://www.webtoons.com/en/drama/elegant-desire/episode-14/viewer?title_no=5836&episode_no=14 65 36.49104619026184
finished https://www.webtoons.com/en/romance/wedding-strategy/episode-16/viewer?title_no=5892&episode_no=16 66 36.98437809944153
finished https://www.webtoons.com/en/action/brave-citizen/episode-29/viewer?title_no=5640&episode_no=30 67 37.4236741065979
finished https://www.webtoons.com/en/drama/the-producers-second-life/episode-6/viewer?title_no=5046&episode_no=6 68 37.83936524391174
finished https://www.webtoons.com/en/action/code-adam/s3-episode-163/viewer?title_no=1657&episode_no=165 69 38.45119023323059
finished https://www.webtoons.com/en/fantasy/the-tyrant-wants-to-be-good/episode-58/viewer?title_no=5201&episode_no=58 70 38.941158294677734
finished https://www.webtoons.com/en/drama/acception/episode-258/viewer?title_no=1513&episode_no=258 71 39.61969017982483
finished https://www.webtoons.com/en/slice-of-life/vampire-husband/s2-ep-148-fits/viewer?title_no=4

ConnectTimeout: 

In [62]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[20:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

finished https://www.webtoons.com/en/romance/my-younger-brothers-friend/episode-5/viewer?title_no=3163&episode_no=6 1 2.034390926361084
finished https://www.webtoons.com/en/thriller/bastard/ep-3/viewer?title_no=485&episode_no=4 2 2.8670618534088135
finished https://www.webtoons.com/en/comedy/mage-and-demon-queen/episode-18/viewer?title_no=1438&episode_no=18 3 3.579832077026367
finished https://www.webtoons.com/en/thriller/its-mine/episode-5/viewer?title_no=2010&episode_no=6 4 4.058125019073486
finished https://www.webtoons.com/en/romance/age-matters/episode-7/viewer?title_no=1364&episode_no=8 5 4.723001956939697
finished https://www.webtoons.com/en/romance/long-after-the-ending/episode-8/viewer?title_no=4179&episode_no=8 6 5.162277936935425
finished https://www.webtoons.com/en/romance/there-must-be-happy-endings/episode-2/viewer?title_no=3065&episode_no=2 7 5.865008115768433
finished https://www.webtoons.com/en/action/a-useless-villain/episode-1/viewer?title_no=2969&episode_no=1 8 6.26

finished https://www.webtoons.com/en/drama/elegant-desire/episode-13/viewer?title_no=5836&episode_no=13 65 48.39324378967285
finished https://www.webtoons.com/en/romance/wedding-strategy/episode-15/viewer?title_no=5892&episode_no=15 66 48.91709589958191
finished https://www.webtoons.com/en/action/brave-citizen/episode-28/viewer?title_no=5640&episode_no=29 67 49.61723303794861
finished https://www.webtoons.com/en/drama/the-producers-second-life/episode-5/viewer?title_no=5046&episode_no=5 68 49.9988579750061
finished https://www.webtoons.com/en/action/code-adam/s3-episode-162/viewer?title_no=1657&episode_no=164 69 50.55377674102783
finished https://www.webtoons.com/en/fantasy/the-tyrant-wants-to-be-good/episode-57/viewer?title_no=5201&episode_no=57 70 51.05051898956299
finished https://www.webtoons.com/en/drama/acception/episode-257/viewer?title_no=1513&episode_no=257 71 51.8034029006958
finished https://www.webtoons.com/en/slice-of-life/vampire-husband/s2-ep-147-mcvamps/viewer?title_no=

ReadTimeout: 

In [63]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[21:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

finished https://www.webtoons.com/en/romance/my-younger-brothers-friend/episode-4/viewer?title_no=3163&episode_no=5 1 1.018347978591919
finished https://www.webtoons.com/en/thriller/bastard/ep-2/viewer?title_no=485&episode_no=3 2 1.58052396774292
finished https://www.webtoons.com/en/comedy/mage-and-demon-queen/episode-17/viewer?title_no=1438&episode_no=17 3 1.995635747909546
finished https://www.webtoons.com/en/thriller/its-mine/episode-4/viewer?title_no=2010&episode_no=5 4 2.6139469146728516
finished https://www.webtoons.com/en/romance/age-matters/episode-6/viewer?title_no=1364&episode_no=7 5 3.364915132522583
finished https://www.webtoons.com/en/romance/long-after-the-ending/episode-7/viewer?title_no=4179&episode_no=7 6 3.746501922607422
finished https://www.webtoons.com/en/romance/there-must-be-happy-endings/episode-1/viewer?title_no=3065&episode_no=1 7 4.396236896514893
finished https://www.webtoons.com/en/thriller/zombie-wife/ep-3-phantom-memory/viewer?title_no=5972&episode_no=3 8

finished https://www.webtoons.com/en/drama/elegant-desire/episode-12/viewer?title_no=5836&episode_no=12 65 47.97200298309326
finished https://www.webtoons.com/en/romance/wedding-strategy/episode-14/viewer?title_no=5892&episode_no=14 66 48.74400997161865
finished https://www.webtoons.com/en/action/brave-citizen/episode-27/viewer?title_no=5640&episode_no=28 67 49.9370379447937
finished https://www.webtoons.com/en/drama/just-one-bite/episode-7/viewer?title_no=5215&episode_no=7 68 50.415343046188354
finished https://www.webtoons.com/en/action/code-adam/s3-episode-161/viewer?title_no=1657&episode_no=163 69 51.19778490066528
finished https://www.webtoons.com/en/fantasy/the-tyrant-wants-to-be-good/episode-56/viewer?title_no=5201&episode_no=56 70 51.702069997787476
finished https://www.webtoons.com/en/drama/acception/episode-256/viewer?title_no=1513&episode_no=256 71 52.68157505989075
finished https://www.webtoons.com/en/slice-of-life/vampire-husband/s2-ep-146-wedding/viewer?title_no=4841&epis

ConnectTimeout: 

In [64]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[22:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

finished https://www.webtoons.com/en/romance/my-younger-brothers-friend/episode-3/viewer?title_no=3163&episode_no=4 1 1.2810490131378174
finished https://www.webtoons.com/en/thriller/bastard/ep-1/viewer?title_no=485&episode_no=2 2 6.65581488609314
finished https://www.webtoons.com/en/comedy/mage-and-demon-queen/episode-16/viewer?title_no=1438&episode_no=16 3 10.191894054412842
finished https://www.webtoons.com/en/thriller/its-mine/episode-3/viewer?title_no=2010&episode_no=4 4 14.581521987915039
finished https://www.webtoons.com/en/romance/age-matters/episode-5/viewer?title_no=1364&episode_no=6 5 16.737854957580566
finished https://www.webtoons.com/en/romance/long-after-the-ending/episode-6/viewer?title_no=4179&episode_no=6 6 20.37237310409546
finished https://www.webtoons.com/en/romance/a-chance-at-last/episode-7/viewer?title_no=3909&episode_no=7 7 22.34686303138733
finished https://www.webtoons.com/en/thriller/zombie-wife/ep-2-test-subject/viewer?title_no=5972&episode_no=2 8 25.455675

finished https://www.webtoons.com/en/comedy/safely-endangered/ep-976-cd-romance/viewer?title_no=352&episode_no=976 64 106.62068605422974
finished https://www.webtoons.com/en/fantasy/lovely-hell/s2-ep-97-destined/viewer?title_no=3181&episode_no=97 65 107.60638213157654
finished https://www.webtoons.com/en/romance/wedding-strategy/episode-13/viewer?title_no=5892&episode_no=13 66 108.62859320640564
finished https://www.webtoons.com/en/action/brave-citizen/episode-26/viewer?title_no=5640&episode_no=27 67 109.75088620185852
finished https://www.webtoons.com/en/drama/just-one-bite/episode-6/viewer?title_no=5215&episode_no=6 68 110.64792609214783
finished https://www.webtoons.com/en/action/code-adam/s3-episode-160/viewer?title_no=1657&episode_no=162 69 111.33467388153076
finished https://www.webtoons.com/en/fantasy/the-tyrant-wants-to-be-good/episode-55/viewer?title_no=5201&episode_no=55 70 111.93316912651062
finished https://www.webtoons.com/en/drama/acception/episode-255/viewer?title_no=151

ConnectTimeout: 

In [65]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[23:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

finished https://www.webtoons.com/en/romance/my-younger-brothers-friend/episode-2/viewer?title_no=3163&episode_no=3 1 2.292056083679199
finished https://www.webtoons.com/en/fantasy/the-first-night-with-the-duke/episode-7/viewer?title_no=1267&episode_no=7 2 2.7902560234069824
finished https://www.webtoons.com/en/comedy/mage-and-demon-queen/episode-15/viewer?title_no=1438&episode_no=15 3 3.201758861541748
finished https://www.webtoons.com/en/thriller/its-mine/episode-2/viewer?title_no=2010&episode_no=3 4 3.844025135040283
finished https://www.webtoons.com/en/romance/age-matters/episode-4/viewer?title_no=1364&episode_no=5 5 4.226824998855591
finished https://www.webtoons.com/en/romance/long-after-the-ending/episode-5/viewer?title_no=4179&episode_no=5 6 4.81575608253479
finished https://www.webtoons.com/en/romance/a-chance-at-last/episode-6/viewer?title_no=3909&episode_no=6 7 5.245596885681152
finished https://www.webtoons.com/en/thriller/zombie-wife/ep-1-life-bites/viewer?title_no=5972&ep

ReadTimeout: 

In [66]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[24:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

finished https://www.webtoons.com/en/romance/my-younger-brothers-friend/episode-1/viewer?title_no=3163&episode_no=2 1 0.988487720489502
finished https://www.webtoons.com/en/fantasy/the-first-night-with-the-duke/episode-6/viewer?title_no=1267&episode_no=6 2 1.9376518726348877
finished https://www.webtoons.com/en/comedy/mage-and-demon-queen/episode-14/viewer?title_no=1438&episode_no=14 3 2.3272018432617188
finished https://www.webtoons.com/en/thriller/its-mine/episode-1/viewer?title_no=2010&episode_no=2 4 2.764936923980713
finished https://www.webtoons.com/en/romance/the-makeup-remover/episode-7/viewer?title_no=2186&episode_no=7 5 3.175194025039673
finished https://www.webtoons.com/en/romance/long-after-the-ending/episode-4/viewer?title_no=4179&episode_no=4 6 3.7322299480438232
finished https://www.webtoons.com/en/romance/a-chance-at-last/episode-5/viewer?title_no=3909&episode_no=5 7 4.150660037994385
finished https://www.webtoons.com/en/fantasy/patience-my-lady/episode-3/viewer?title_no

finished https://www.webtoons.com/en/fantasy/lovely-hell/s2-ep-95-it-wont-matter/viewer?title_no=3181&episode_no=95 65 41.80090379714966
finished https://www.webtoons.com/en/romance/wedding-strategy/episode-11/viewer?title_no=5892&episode_no=11 66 42.480560064315796
finished https://www.webtoons.com/en/action/brave-citizen/episode-24/viewer?title_no=5640&episode_no=25 67 43.08829689025879
finished https://www.webtoons.com/en/drama/just-one-bite/episode-4/viewer?title_no=5215&episode_no=4 68 43.47747182846069
finished https://www.webtoons.com/en/action/code-adam/s3-episode-158/viewer?title_no=1657&episode_no=160 69 44.38456177711487
finished https://www.webtoons.com/en/fantasy/the-tyrant-wants-to-be-good/episode-53/viewer?title_no=5201&episode_no=53 70 45.463768005371094
finished https://www.webtoons.com/en/drama/acception/episode-253/viewer?title_no=1513&episode_no=253 71 46.466962814331055
finished https://www.webtoons.com/en/slice-of-life/vampire-husband/s2-ep-143-fetch/viewer?title_

ReadTimeout: 

In [67]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[25:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

finished https://www.webtoons.com/en/romance/my-younger-brothers-friend/episode-0-prologue/viewer?title_no=3163&episode_no=1 1 1.1698579788208008
finished https://www.webtoons.com/en/fantasy/the-first-night-with-the-duke/episode-5/viewer?title_no=1267&episode_no=5 2 1.9820070266723633
finished https://www.webtoons.com/en/comedy/mage-and-demon-queen/episode-13-melathia-fanfic-2/viewer?title_no=1438&episode_no=13 3 2.8868210315704346
finished https://www.webtoons.com/en/thriller/its-mine/episode-0-prologue/viewer?title_no=2010&episode_no=1 4 3.6021251678466797
finished https://www.webtoons.com/en/romance/the-makeup-remover/episode-6/viewer?title_no=2186&episode_no=6 5 4.0883629322052
finished https://www.webtoons.com/en/romance/long-after-the-ending/episode-3/viewer?title_no=4179&episode_no=3 6 4.623962163925171
finished https://www.webtoons.com/en/romance/a-chance-at-last/episode-4/viewer?title_no=3909&episode_no=4 7 5.216466188430786
finished https://www.webtoons.com/en/fantasy/patienc

/Users/zacharymeurer/opt/anaconda3/lib/python3.9/site-packages/bs4/builder/__init__.py:314: RuntimeWarning: coroutine 'download_image' was never awaited
  for attr in list(attrs.keys()):


finished https://www.webtoons.com/en/romance/romantic-graffiti/episode-3/viewer?title_no=5049&episode_no=3 12 9.206600904464722
finished https://www.webtoons.com/en/romance/half-ghost/episode-6/viewer?title_no=2765&episode_no=7 13 10.446493148803711
finished https://www.webtoons.com/en/thriller/tales-of-greed/ep-2-face-correction-2/viewer?title_no=2772&episode_no=2 14 11.85680103302002
finished https://www.webtoons.com/en/romance/time-and-reason/episode-6/viewer?title_no=4210&episode_no=6 15 13.34836196899414
finished https://www.webtoons.com/en/thriller/hell-is-other-people/episode-8/viewer?title_no=3176&episode_no=8 16 14.531153202056885
finished https://www.webtoons.com/en/romance/her-bucket-list/episode-5/viewer?title_no=2564&episode_no=5 17 15.00882887840271
finished https://www.webtoons.com/en/fantasy/flow/ep-2/viewer?title_no=101&episode_no=3 18 15.670614957809448
finished https://www.webtoons.com/en/action/deadlife/episode-3/viewer?title_no=3922&episode_no=4 19 16.3388841152191

finished https://www.webtoons.com/en/action/soul-gear/s2-episode-44/viewer?title_no=4473&episode_no=44 75 57.37167716026306
finished https://www.webtoons.com/en/romance/30-minutes-with-you/ep-6-bothered/viewer?title_no=5231&episode_no=6 76 58.041829109191895
finished https://www.webtoons.com/en/horror/after-dark/episode-26/viewer?title_no=5110&episode_no=26 77 58.63495397567749
finished https://www.webtoons.com/en/fantasy/nothing-special/s3-episode-64/viewer?title_no=1188&episode_no=151 78 59.249622106552124
finished https://www.webtoons.com/en/slice-of-life/daybreak/s2-episode-135/viewer?title_no=4089&episode_no=135 79 59.95663905143738
finished https://www.webtoons.com/en/action/my-s-class-hunters/ep-105-old-future-enemy/viewer?title_no=3963&episode_no=105 80 60.693289041519165
finished https://www.webtoons.com/en/historical/return-of-the-mad-demon/episode-98/viewer?title_no=3671&episode_no=98 81 61.417729139328
finished https://www.webtoons.com/en/sf/rebirth/s3-episode-220/viewer?ti

ConnectTimeout: 

In [68]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[26:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

finished https://www.webtoons.com/en/romance/the-return-of-elena-the-knight/episode-3/viewer?title_no=5720&episode_no=3 1 1.2878026962280273
finished https://www.webtoons.com/en/fantasy/the-first-night-with-the-duke/episode-4/viewer?title_no=1267&episode_no=4 2 2.01674485206604
finished https://www.webtoons.com/en/comedy/mage-and-demon-queen/episode-12-melathia-fanfic-1/viewer?title_no=1438&episode_no=12 3 2.643439769744873
finished https://www.webtoons.com/en/romance/the-hidden-saintess/episode-3/viewer?title_no=4114&episode_no=3 4 3.4057610034942627
finished https://www.webtoons.com/en/romance/the-makeup-remover/episode-5/viewer?title_no=2186&episode_no=5 5 4.258408784866333
finished https://www.webtoons.com/en/romance/long-after-the-ending/episode-2/viewer?title_no=4179&episode_no=2 6 4.627408742904663
finished https://www.webtoons.com/en/romance/a-chance-at-last/episode-3/viewer?title_no=3909&episode_no=3 7 4.976982831954956
finished https://www.webtoons.com/en/fantasy/patience-my-

finished https://www.webtoons.com/en/comedy/safely-endangered/ep-972-spiders/viewer?title_no=352&episode_no=972 64 44.34999489784241
finished https://www.webtoons.com/en/fantasy/lovely-hell/s2-ep-93-traitors/viewer?title_no=3181&episode_no=93 65 45.39271283149719
finished https://www.webtoons.com/en/romance/wedding-strategy/episode-9/viewer?title_no=5892&episode_no=9 66 45.756372928619385
finished https://www.webtoons.com/en/action/brave-citizen/episode-22/viewer?title_no=5640&episode_no=23 67 46.44278287887573
finished https://www.webtoons.com/en/drama/just-one-bite/episode-2/viewer?title_no=5215&episode_no=2 68 47.275529861450195
finished https://www.webtoons.com/en/action/code-adam/s3-episode-156/viewer?title_no=1657&episode_no=158 69 48.66619372367859
finished https://www.webtoons.com/en/fantasy/the-tyrant-wants-to-be-good/episode-51/viewer?title_no=5201&episode_no=51 70 49.18995785713196
finished https://www.webtoons.com/en/drama/acception/episode-251/viewer?title_no=1513&episode_

In [70]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[27:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

finished https://www.webtoons.com/en/romance/the-return-of-elena-the-knight/episode-2/viewer?title_no=5720&episode_no=2 1 3.394461154937744
finished https://www.webtoons.com/en/fantasy/the-first-night-with-the-duke/episode-3/viewer?title_no=1267&episode_no=3 2 3.8455569744110107
finished https://www.webtoons.com/en/comedy/mage-and-demon-queen/episode-11/viewer?title_no=1438&episode_no=11 3 4.945274114608765
finished https://www.webtoons.com/en/romance/the-hidden-saintess/episode-2/viewer?title_no=4114&episode_no=2 4 5.318681001663208
finished https://www.webtoons.com/en/romance/the-makeup-remover/episode-4/viewer?title_no=2186&episode_no=4 5 5.8235838413238525
finished https://www.webtoons.com/en/romance/long-after-the-ending/episode-1/viewer?title_no=4179&episode_no=1 6 6.372174978256226
finished https://www.webtoons.com/en/romance/a-chance-at-last/episode-2/viewer?title_no=3909&episode_no=2 7 6.879053115844727
finished https://www.webtoons.com/en/romance/rumor-has-it/episode-10/viewe

finished https://www.webtoons.com/en/comedy/safely-endangered/ep-971-clever-grill/viewer?title_no=352&episode_no=971 64 40.88782000541687
finished https://www.webtoons.com/en/fantasy/lovely-hell/s2-ep-92-dont-be-stubborn/viewer?title_no=3181&episode_no=92 65 41.37178993225098
finished https://www.webtoons.com/en/romance/wedding-strategy/episode-8/viewer?title_no=5892&episode_no=8 66 41.81328296661377
finished https://www.webtoons.com/en/action/brave-citizen/episode-21/viewer?title_no=5640&episode_no=22 67 42.28888487815857
finished https://www.webtoons.com/en/drama/just-one-bite/episode-1/viewer?title_no=5215&episode_no=1 68 42.8815701007843
finished https://www.webtoons.com/en/drama/fictional-skin/episode-47-season-finale/viewer?title_no=3272&episode_no=47 69 43.43563890457153
finished https://www.webtoons.com/en/fantasy/the-tyrant-wants-to-be-good/episode-50/viewer?title_no=5201&episode_no=50 70 43.85047793388367
finished https://www.webtoons.com/en/drama/acception/episode-250/viewer

ConnectTimeout: 

In [71]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[28:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

finished https://www.webtoons.com/en/romance/the-return-of-elena-the-knight/episode-1/viewer?title_no=5720&episode_no=1 1 1.1304380893707275
finished https://www.webtoons.com/en/fantasy/the-first-night-with-the-duke/episode-2/viewer?title_no=1267&episode_no=2 2 1.8563661575317383
finished https://www.webtoons.com/en/comedy/mage-and-demon-queen/episode-10/viewer?title_no=1438&episode_no=10 3 2.2843902111053467
finished https://www.webtoons.com/en/romance/the-hidden-saintess/episode-1/viewer?title_no=4114&episode_no=1 4 2.7103030681610107
finished https://www.webtoons.com/en/romance/the-makeup-remover/episode-3/viewer?title_no=2186&episode_no=3 5 3.1346230506896973
finished https://www.webtoons.com/en/romance/cierra/episode-7/viewer?title_no=3670&episode_no=7 6 3.5439541339874268
finished https://www.webtoons.com/en/romance/a-chance-at-last/episode-1/viewer?title_no=3909&episode_no=1 7 3.9410359859466553
finished https://www.webtoons.com/en/romance/rumor-has-it/episode-9/viewer?title_no=

finished https://www.webtoons.com/en/romance/blood-reverie/s2-episode-71/viewer?title_no=2806&episode_no=72 63 35.25039625167847
finished https://www.webtoons.com/en/comedy/safely-endangered/ep-970-metasquad/viewer?title_no=352&episode_no=970 64 36.474493980407715
finished https://www.webtoons.com/en/fantasy/lovely-hell/s2-ep-91-the-gift/viewer?title_no=3181&episode_no=91 65 37.01688504219055
finished https://www.webtoons.com/en/romance/wedding-strategy/episode-7/viewer?title_no=5892&episode_no=7 66 37.83718705177307
finished https://www.webtoons.com/en/action/brave-citizen/episode-20/viewer?title_no=5640&episode_no=21 67 38.33712720870972
finished https://www.webtoons.com/en/romance/my-lovely-bodyguard/episode-7/viewer?title_no=5191&episode_no=7 68 38.854856967926025
finished https://www.webtoons.com/en/drama/fictional-skin/episode-46/viewer?title_no=3272&episode_no=46 69 39.625138998031616
finished https://www.webtoons.com/en/fantasy/the-tyrant-wants-to-be-good/episode-49/viewer?titl

ConnectTimeout: 

In [72]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[29:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

finished https://www.webtoons.com/en/supernatural/unholy-blood/episode-12/viewer?title_no=1262&episode_no=13 1 3.0648491382598877
finished https://www.webtoons.com/en/fantasy/the-first-night-with-the-duke/episode-1/viewer?title_no=1267&episode_no=1 2 3.837409019470215
finished https://www.webtoons.com/en/romance/my-far-too-tumultuous-marriage/episode-7/viewer?title_no=5230&episode_no=7 3 4.205878973007202
finished https://www.webtoons.com/en/romance/lemon-soda-and-coffee/ep-15-golden-trio/viewer?title_no=5971&episode_no=15 4 4.716568946838379
finished https://www.webtoons.com/en/romance/the-makeup-remover/episode-2/viewer?title_no=2186&episode_no=2 5 5.724295139312744
finished https://www.webtoons.com/en/romance/cierra/episode-6/viewer?title_no=3670&episode_no=6 6 6.06401801109314
finished https://www.webtoons.com/en/romance/miss-abbott-and-the-doctor/ep-17-who-knows/viewer?title_no=707&episode_no=20 7 6.813450813293457
finished https://www.webtoons.com/en/romance/rumor-has-it/episode-

finished https://www.webtoons.com/en/comedy/safely-endangered/ep-969-pumped/viewer?title_no=352&episode_no=969 64 40.81906080245972
finished https://www.webtoons.com/en/fantasy/lovely-hell/s2-ep-90-the-choice/viewer?title_no=3181&episode_no=90 65 41.34842801094055
finished https://www.webtoons.com/en/sports/the-build-up/episode-156/viewer?title_no=2712&episode_no=157 66 42.0306339263916
finished https://www.webtoons.com/en/fantasy/the-fabled-warrior/s2-episode-143/viewer?title_no=3179&episode_no=143 67 42.48725605010986
finished https://www.webtoons.com/en/romance/my-lovely-bodyguard/episode-6/viewer?title_no=5191&episode_no=6 68 43.019171953201294
finished https://www.webtoons.com/en/drama/fictional-skin/episode-45/viewer?title_no=3272&episode_no=45 69 43.60632276535034
finished https://www.webtoons.com/en/slice-of-life/boyfriends/s2-episode-196/viewer?title_no=2616&episode_no=196 70 44.25429892539978
finished https://www.webtoons.com/en/comedy/stalker-x-stalker/episode-15/viewer?titl

ReadTimeout: 

In [73]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[30:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

finished https://www.webtoons.com/en/supernatural/unholy-blood/episode-11/viewer?title_no=1262&episode_no=12 1 1.0569710731506348
finished https://www.webtoons.com/en/fantasy/dark-moon-the-blood-altar/6-the-wolves-who-jumped-over-the-fence/viewer?title_no=3785&episode_no=6 2 1.476741075515747
finished https://www.webtoons.com/en/romance/my-far-too-tumultuous-marriage/episode-6/viewer?title_no=5230&episode_no=6 3 2.0447869300842285
finished https://www.webtoons.com/en/romance/lemon-soda-and-coffee/ep-14-enjoy/viewer?title_no=5971&episode_no=14 4 3.949737071990967
finished https://www.webtoons.com/en/romance/the-makeup-remover/episode-1/viewer?title_no=2186&episode_no=1 5 4.881239175796509
finished https://www.webtoons.com/en/romance/cierra/episode-5/viewer?title_no=3670&episode_no=5 6 5.410187005996704
finished https://www.webtoons.com/en/romance/miss-abbott-and-the-doctor/ep-16-the-note-/viewer?title_no=707&episode_no=19 7 5.972935914993286
finished https://www.webtoons.com/en/romance/

finished https://www.webtoons.com/en/comedy/safely-endangered/ep-968-1-dad/viewer?title_no=352&episode_no=968 64 44.02737307548523
finished https://www.webtoons.com/en/fantasy/lovely-hell/s2-ep-89-poison/viewer?title_no=3181&episode_no=89 65 44.630720138549805
finished https://www.webtoons.com/en/sports/the-build-up/episode-155/viewer?title_no=2712&episode_no=156 66 45.196582078933716
finished https://www.webtoons.com/en/fantasy/the-fabled-warrior/s2-episode-142/viewer?title_no=3179&episode_no=142 67 45.67648720741272
finished https://www.webtoons.com/en/romance/my-lovely-bodyguard/episode-5/viewer?title_no=5191&episode_no=5 68 46.55260419845581
finished https://www.webtoons.com/en/drama/fictional-skin/episode-44/viewer?title_no=3272&episode_no=44 69 46.95373606681824
finished https://www.webtoons.com/en/slice-of-life/boyfriends/s2-episode-195/viewer?title_no=2616&episode_no=195 70 47.76133394241333
finished https://www.webtoons.com/en/comedy/stalker-x-stalker/episode-14/viewer?title_n

ReadTimeout: 

In [74]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[31:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

finished https://www.webtoons.com/en/supernatural/unholy-blood/episode-10/viewer?title_no=1262&episode_no=11 1 1.8510057926177979
finished https://www.webtoons.com/en/fantasy/dark-moon-the-blood-altar/5-clean-the-room-and-reveal-the-big-secret/viewer?title_no=3785&episode_no=5 2 2.3673288822174072
finished https://www.webtoons.com/en/romance/my-far-too-tumultuous-marriage/episode-5/viewer?title_no=5230&episode_no=5 3 4.245649099349976
finished https://www.webtoons.com/en/romance/lemon-soda-and-coffee/ep-13-too-much/viewer?title_no=5971&episode_no=13 4 5.204991817474365
finished https://www.webtoons.com/en/drama/mystic-prince/ep-15-traces-that-remain/viewer?title_no=4737&episode_no=15 5 5.809160947799683
finished https://www.webtoons.com/en/romance/cierra/episode-4/viewer?title_no=3670&episode_no=4 6 6.81529688835144
finished https://www.webtoons.com/en/romance/miss-abbott-and-the-doctor/ep-15-tell-me-about-cati/viewer?title_no=707&episode_no=18 7 7.2827630043029785
finished https://www

finished https://www.webtoons.com/en/romance/blood-reverie/s2-episode-68/viewer?title_no=2806&episode_no=69 63 40.703887939453125
finished https://www.webtoons.com/en/comedy/safely-endangered/ep-967-i-choose-me/viewer?title_no=352&episode_no=967 64 42.50904893875122
finished https://www.webtoons.com/en/fantasy/lovely-hell/s2-ep-88-theres-still-time/viewer?title_no=3181&episode_no=88 65 42.95208692550659
finished https://www.webtoons.com/en/sports/the-build-up/episode-154/viewer?title_no=2712&episode_no=155 66 43.400582790374756
finished https://www.webtoons.com/en/fantasy/the-fabled-warrior/s2-episode-141/viewer?title_no=3179&episode_no=141 67 44.335917949676514
finished https://www.webtoons.com/en/romance/my-lovely-bodyguard/episode-4-/viewer?title_no=5191&episode_no=4 68 45.187185764312744
finished https://www.webtoons.com/en/drama/fictional-skin/episode-43/viewer?title_no=3272&episode_no=43 69 45.80779194831848
finished https://www.webtoons.com/en/slice-of-life/boyfriends/s2-episode

ReadTimeout: 

In [75]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[32:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

finished https://www.webtoons.com/en/supernatural/unholy-blood/episode-9/viewer?title_no=1262&episode_no=10 1 0.7829480171203613
finished https://www.webtoons.com/en/fantasy/dark-moon-the-blood-altar/4-one-step-back-two-steps-forward/viewer?title_no=3785&episode_no=4 2 1.9329609870910645
finished https://www.webtoons.com/en/romance/my-far-too-tumultuous-marriage/episode-4/viewer?title_no=5230&episode_no=4 3 4.719740867614746
finished https://www.webtoons.com/en/romance/lemon-soda-and-coffee/ep-12-similar/viewer?title_no=5971&episode_no=12 4 5.3912577629089355
finished https://www.webtoons.com/en/drama/mystic-prince/ep-14-about-that-night/viewer?title_no=4737&episode_no=14 5 6.654845952987671
finished https://www.webtoons.com/en/romance/cierra/episode-3/viewer?title_no=3670&episode_no=3 6 8.31474781036377
finished https://www.webtoons.com/en/romance/miss-abbott-and-the-doctor/ep-14-completely-cured/viewer?title_no=707&episode_no=17 7 8.800666809082031
finished https://www.webtoons.com/e

finished https://www.webtoons.com/en/romance/blood-reverie/s2-episode-67/viewer?title_no=2806&episode_no=68 63 43.36395001411438
finished https://www.webtoons.com/en/action/hectopascal/s2-episode-63/viewer?title_no=5033&episode_no=63 64 44.096380949020386
finished https://www.webtoons.com/en/thriller/class-1-9/afterword/viewer?title_no=4955&episode_no=55 65 44.65846490859985
finished https://www.webtoons.com/en/sports/the-build-up/episode-153/viewer?title_no=2712&episode_no=154 66 45.133460998535156
finished https://www.webtoons.com/en/fantasy/the-fabled-warrior/s2-episode-140/viewer?title_no=3179&episode_no=140 67 45.73846101760864
finished https://www.webtoons.com/en/romance/my-lovely-bodyguard/episode-3/viewer?title_no=5191&episode_no=3 68 46.11434292793274
finished https://www.webtoons.com/en/drama/fictional-skin/episode-42/viewer?title_no=3272&episode_no=42 69 46.54845380783081
finished https://www.webtoons.com/en/slice-of-life/boyfriends/s2-episode-193/viewer?title_no=2616&episod

ConnectTimeout: 

In [76]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[33:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

finished https://www.webtoons.com/en/supernatural/unholy-blood/episode-8/viewer?title_no=1262&episode_no=9 1 2.3224148750305176
finished https://www.webtoons.com/en/fantasy/dark-moon-the-blood-altar/3-its-nice-to-meet-you/viewer?title_no=3785&episode_no=3 2 2.7868330478668213
finished https://www.webtoons.com/en/romance/my-far-too-tumultuous-marriage/episode-3/viewer?title_no=5230&episode_no=3 3 3.394644021987915
finished https://www.webtoons.com/en/romance/lemon-soda-and-coffee/ep-11-struggle/viewer?title_no=5971&episode_no=11 4 16.682034969329834
finished https://www.webtoons.com/en/drama/mystic-prince/ep-13-the-reprimand/viewer?title_no=4737&episode_no=13 5 17.08024001121521
finished https://www.webtoons.com/en/romance/cierra/episode-2/viewer?title_no=3670&episode_no=2 6 17.492083072662354
finished https://www.webtoons.com/en/romance/miss-abbott-and-the-doctor/ep-13b-tachycardia-cati/viewer?title_no=707&episode_no=16 7 21.0700581073761
finished https://www.webtoons.com/en/romance/ru

finished https://www.webtoons.com/en/romance/blood-reverie/s2-episode-66/viewer?title_no=2806&episode_no=67 63 56.875569105148315
finished https://www.webtoons.com/en/action/hectopascal/s2-episode-62/viewer?title_no=5033&episode_no=62 64 57.46946907043457
finished https://www.webtoons.com/en/thriller/class-1-9/episode-54-series-finale/viewer?title_no=4955&episode_no=54 65 58.07378911972046
finished https://www.webtoons.com/en/sports/the-build-up/episode-152/viewer?title_no=2712&episode_no=153 66 58.59222602844238
finished https://www.webtoons.com/en/fantasy/the-fabled-warrior/s2-episode-139/viewer?title_no=3179&episode_no=139 67 59.04657602310181
finished https://www.webtoons.com/en/romance/my-lovely-bodyguard/episode-2/viewer?title_no=5191&episode_no=2 68 59.41378116607666
finished https://www.webtoons.com/en/drama/fictional-skin/episode-41/viewer?title_no=3272&episode_no=41 69 59.78410792350769
finished https://www.webtoons.com/en/slice-of-life/boyfriends/s2-episode-192/viewer?title_

In [77]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[34:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

finished https://www.webtoons.com/en/supernatural/unholy-blood/episode-7/viewer?title_no=1262&episode_no=8 1 3.601224899291992
finished https://www.webtoons.com/en/fantasy/dark-moon-the-blood-altar/2-hello-heli/viewer?title_no=3785&episode_no=2 2 3.9900050163269043
finished https://www.webtoons.com/en/romance/my-far-too-tumultuous-marriage/episode-2/viewer?title_no=5230&episode_no=2 3 4.4025559425354
finished https://www.webtoons.com/en/romance/lemon-soda-and-coffee/ep-10-nightmare/viewer?title_no=5971&episode_no=10 4 4.806594133377075
finished https://www.webtoons.com/en/drama/mystic-prince/ep-12-the-ghost-bride-5/viewer?title_no=4737&episode_no=12 5 5.173723936080933
finished https://www.webtoons.com/en/romance/cierra/episode-1/viewer?title_no=3670&episode_no=1 6 5.5361008644104
finished https://www.webtoons.com/en/romance/miss-abbott-and-the-doctor/ep-13a-tachycardia-andreas-/viewer?title_no=707&episode_no=15 7 5.9228010177612305
finished https://www.webtoons.com/en/romance/rumor-ha

finished https://www.webtoons.com/en/action/hectopascal/s2-episode-61/viewer?title_no=5033&episode_no=61 64 32.10623478889465
finished https://www.webtoons.com/en/thriller/class-1-9/episode-53/viewer?title_no=4955&episode_no=53 65 32.592402935028076
finished https://www.webtoons.com/en/sports/the-build-up/episode-151/viewer?title_no=2712&episode_no=152 66 33.23433709144592
finished https://www.webtoons.com/en/fantasy/the-fabled-warrior/s2-episode-138/viewer?title_no=3179&episode_no=138 67 33.89002013206482
finished https://www.webtoons.com/en/romance/my-lovely-bodyguard/episode-1/viewer?title_no=5191&episode_no=1 68 34.238879919052124
finished https://www.webtoons.com/en/drama/fictional-skin/episode-40/viewer?title_no=3272&episode_no=40 69 34.72755789756775
finished https://www.webtoons.com/en/slice-of-life/boyfriends/s2-episode-191/viewer?title_no=2616&episode_no=191 70 35.19560980796814
finished https://www.webtoons.com/en/comedy/stalker-x-stalker/episode-10/viewer?title_no=6080&epis

ReadTimeout: 

In [78]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[35:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

finished https://www.webtoons.com/en/supernatural/unholy-blood/episode-6/viewer?title_no=1262&episode_no=7 1 1.3949131965637207
finished https://www.webtoons.com/en/fantasy/dark-moon-the-blood-altar/1-the-first-encounter/viewer?title_no=3785&episode_no=1 2 1.9249942302703857
finished https://www.webtoons.com/en/romance/my-far-too-tumultuous-marriage/episode-1/viewer?title_no=5230&episode_no=1 3 2.336942195892334
finished https://www.webtoons.com/en/romance/lemon-soda-and-coffee/ep-9-big-steps/viewer?title_no=5971&episode_no=9 4 2.7713911533355713
finished https://www.webtoons.com/en/drama/mystic-prince/ep-11-the-ghost-bride-4/viewer?title_no=4737&episode_no=11 5 15.734944105148315
finished https://www.webtoons.com/en/action/the-first-hunter/episode-5/viewer?title_no=3040&episode_no=6 6 16.26484203338623
finished https://www.webtoons.com/en/romance/miss-abbott-and-the-doctor/ep-12-he-doesnt-like-you-either-part-3-/viewer?title_no=707&episode_no=14 7 16.76448917388916
finished https://ww

finished https://www.webtoons.com/en/romance/blood-reverie/s2-episode-64/viewer?title_no=2806&episode_no=65 63 44.23795008659363
finished https://www.webtoons.com/en/action/hectopascal/s2-episode-60/viewer?title_no=5033&episode_no=60 64 44.84045219421387
finished https://www.webtoons.com/en/thriller/class-1-9/episode-52/viewer?title_no=4955&episode_no=52 65 45.24347114562988
finished https://www.webtoons.com/en/sports/the-build-up/episode-150/viewer?title_no=2712&episode_no=151 66 45.89132905006409
finished https://www.webtoons.com/en/fantasy/the-fabled-warrior/s2-episode-137/viewer?title_no=3179&episode_no=137 67 46.32827019691467
finished https://www.webtoons.com/en/tiptoon/webtoon-now/april-2024-returns/viewer?title_no=2025&episode_no=271 68 46.8763153553009
finished https://www.webtoons.com/en/drama/fictional-skin/episode-39/viewer?title_no=3272&episode_no=39 69 47.30024719238281
finished https://www.webtoons.com/en/slice-of-life/boyfriends/s2-episode-190/viewer?title_no=2616&episo

ConnectError: [Errno 8] nodename nor servname provided, or not known

In [79]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[36:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

finished https://www.webtoons.com/en/supernatural/unholy-blood/episode-5/viewer?title_no=1262&episode_no=6 1 2.9958250522613525
finished https://www.webtoons.com/en/romance/truebeauty/episode-7/viewer?title_no=1436&episode_no=8 2 3.3967959880828857
finished https://www.webtoons.com/en/romance/to-love-your-enemy/episode-12/viewer?title_no=2114&episode_no=12 3 3.836427927017212
finished https://www.webtoons.com/en/romance/lemon-soda-and-coffee/ep-8-bittersweet-memories/viewer?title_no=5971&episode_no=8 4 4.248836040496826
finished https://www.webtoons.com/en/drama/mystic-prince/ep-10-the-ghost-bride-3/viewer?title_no=4737&episode_no=10 5 4.631103992462158
finished https://www.webtoons.com/en/action/the-first-hunter/episode-4/viewer?title_no=3040&episode_no=5 6 5.348978042602539
finished https://www.webtoons.com/en/romance/miss-abbott-and-the-doctor/ep-11-he-doesnt-like-you-either-part-2/viewer?title_no=707&episode_no=13 7 5.791068077087402
finished https://www.webtoons.com/en/romance/rum

finished https://www.webtoons.com/en/romance/blood-reverie/s2-episode-63/viewer?title_no=2806&episode_no=64 63 35.09828805923462
finished https://www.webtoons.com/en/action/hectopascal/s2-episode-59/viewer?title_no=5033&episode_no=59 64 35.49907994270325
finished https://www.webtoons.com/en/thriller/class-1-9/episode-51/viewer?title_no=4955&episode_no=51 65 36.09652090072632
finished https://www.webtoons.com/en/sports/the-build-up/episode-149/viewer?title_no=2712&episode_no=150 66 36.67747187614441
finished https://www.webtoons.com/en/fantasy/the-fabled-warrior/s2-episode-136/viewer?title_no=3179&episode_no=136 67 37.122376918792725
finished https://www.webtoons.com/en/tiptoon/webtoon-now/march-2024-returns/viewer?title_no=2025&episode_no=270 68 38.80379009246826
finished https://www.webtoons.com/en/fantasy/here-there-be-dragons/episode-56-season-finale/viewer?title_no=2726&episode_no=56 69 39.284016132354736
finished https://www.webtoons.com/en/slice-of-life/boyfriends/s2-episode-189/

In [80]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[37:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

finished https://www.webtoons.com/en/supernatural/unholy-blood/episode-4/viewer?title_no=1262&episode_no=5 1 1.1668307781219482
finished https://www.webtoons.com/en/romance/truebeauty/episode-6/viewer?title_no=1436&episode_no=7 2 1.5638747215270996
finished https://www.webtoons.com/en/romance/to-love-your-enemy/episode-11/viewer?title_no=2114&episode_no=11 3 1.9416630268096924
finished https://www.webtoons.com/en/romance/lemon-soda-and-coffee/ep-7-cookies/viewer?title_no=5971&episode_no=7 4 2.3409059047698975
finished https://www.webtoons.com/en/drama/mystic-prince/ep-9-the-ghost-bride-2-/viewer?title_no=4737&episode_no=9 5 2.771224021911621
finished https://www.webtoons.com/en/action/the-first-hunter/episode-3/viewer?title_no=3040&episode_no=4 6 3.1670777797698975
finished https://www.webtoons.com/en/romance/miss-abbott-and-the-doctor/recap/viewer?title_no=707&episode_no=12 7 3.7404990196228027
finished https://www.webtoons.com/en/thriller/shotgun-boy/ep-6/viewer?title_no=2534&episode

finished https://www.webtoons.com/en/romance/as-if-love-doesnt-exist/episode-43-s1-finale/viewer?title_no=5443&episode_no=43 63 30.946151971817017
finished https://www.webtoons.com/en/action/hectopascal/s2-episode-58/viewer?title_no=5033&episode_no=58 64 31.35779571533203
finished https://www.webtoons.com/en/thriller/class-1-9/episode-50/viewer?title_no=4955&episode_no=50 65 31.855741024017334
finished https://www.webtoons.com/en/sports/the-build-up/episode-148/viewer?title_no=2712&episode_no=149 66 32.36114692687988
finished https://www.webtoons.com/en/fantasy/the-fabled-warrior/s2-episode-135/viewer?title_no=3179&episode_no=135 67 33.13331985473633
finished https://www.webtoons.com/en/tiptoon/webtoon-now/february-2024-returns/viewer?title_no=2025&episode_no=269 68 33.75869011878967
finished https://www.webtoons.com/en/fantasy/here-there-be-dragons/episode-55/viewer?title_no=2726&episode_no=55 69 34.287768840789795
finished https://www.webtoons.com/en/slice-of-life/boyfriends/s2-episo

ConnectError: [Errno 8] nodename nor servname provided, or not known

In [81]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[38:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

finished https://www.webtoons.com/en/supernatural/unholy-blood/episode-3/viewer?title_no=1262&episode_no=4 1 6.19536280632019
finished https://www.webtoons.com/en/romance/truebeauty/episode-5/viewer?title_no=1436&episode_no=6 2 6.585216999053955
finished https://www.webtoons.com/en/romance/to-love-your-enemy/episode-10/viewer?title_no=2114&episode_no=10 3 6.976931810379028
finished https://www.webtoons.com/en/romance/lemon-soda-and-coffee/ep-6-two-peas-in-a-pod/viewer?title_no=5971&episode_no=6 4 7.632408857345581
finished https://www.webtoons.com/en/drama/mystic-prince/ep-8-the-ghost-bride-1-/viewer?title_no=4737&episode_no=8 5 8.029464960098267
finished https://www.webtoons.com/en/action/the-first-hunter/episode-2/viewer?title_no=3040&episode_no=3 6 8.406677007675171
finished https://www.webtoons.com/en/romance/miss-abbott-and-the-doctor/ep-10-he-doesnt-like-you-either-part-1-/viewer?title_no=707&episode_no=11 7 8.873731851577759
finished https://www.webtoons.com/en/thriller/shotgun-

finished https://www.webtoons.com/en/romance/as-if-love-doesnt-exist/episode-42/viewer?title_no=5443&episode_no=42 63 36.83625793457031
finished https://www.webtoons.com/en/action/hectopascal/s2-episode-57/viewer?title_no=5033&episode_no=57 64 37.241020917892456
finished https://www.webtoons.com/en/thriller/class-1-9/episode-49/viewer?title_no=4955&episode_no=49 65 37.658406019210815
finished https://www.webtoons.com/en/sports/the-build-up/episode-147/viewer?title_no=2712&episode_no=148 66 38.09270000457764
finished https://www.webtoons.com/en/fantasy/the-fabled-warrior/s2-episode-134/viewer?title_no=3179&episode_no=134 67 38.619723081588745
finished https://www.webtoons.com/en/tiptoon/webtoon-now/january-2024-returns/viewer?title_no=2025&episode_no=268 68 39.266358852386475
finished https://www.webtoons.com/en/fantasy/here-there-be-dragons/episode-54/viewer?title_no=2726&episode_no=54 69 39.814621925354004
finished https://www.webtoons.com/en/slice-of-life/boyfriends/s2-episode-187/vi

ReadTimeout: 

In [82]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[39:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

finished https://www.webtoons.com/en/romance/so-i-married-the-antifan/episode-6/viewer?title_no=2739&episode_no=7 1 1.095022201538086
finished https://www.webtoons.com/en/romance/truebeauty/episode-4/viewer?title_no=1436&episode_no=5 2 1.6252741813659668
finished https://www.webtoons.com/en/romance/to-love-your-enemy/episode-9/viewer?title_no=2114&episode_no=9 3 2.1496620178222656
finished https://www.webtoons.com/en/action/i-get-stronger-the-more-i-eat/episode-4/viewer?title_no=3199&episode_no=5 4 2.6747283935546875
finished https://www.webtoons.com/en/drama/mystic-prince/ep-7-the-second-message-/viewer?title_no=4737&episode_no=7 5 3.1837570667266846
finished https://www.webtoons.com/en/action/the-first-hunter/episode-1/viewer?title_no=3040&episode_no=2 6 3.6870741844177246
finished https://www.webtoons.com/en/romance/nice-to-meet-you/episode-9/viewer?title_no=2066&episode_no=9 7 4.1476452350616455
finished https://www.webtoons.com/en/thriller/shotgun-boy/ep-4/viewer?title_no=2534&epi

finished https://www.webtoons.com/en/action/hectopascal/s2-episode-56/viewer?title_no=5033&episode_no=56 64 37.92051100730896
finished https://www.webtoons.com/en/thriller/class-1-9/episode-48/viewer?title_no=4955&episode_no=48 65 38.38595724105835
finished https://www.webtoons.com/en/horror/webtoon-at-midnight/series-finale-i-discovered-a-town-where-i-could-finally-be-happy-with-family-but-then-their-behavior-took-a-strange-turn-44/viewer?title_no=5773&episode_no=31 66 38.9607093334198
finished https://www.webtoons.com/en/action/mr-baek/ep-7-the-rat/viewer?title_no=5437&episode_no=7 67 39.54476594924927
finished https://www.webtoons.com/en/tiptoon/webtoon-now/november-launch-week-2023/viewer?title_no=2025&episode_no=267 68 40.22338819503784
finished https://www.webtoons.com/en/fantasy/here-there-be-dragons/episode-53/viewer?title_no=2726&episode_no=53 69 40.82392930984497
finished https://www.webtoons.com/en/action/eleceed/episode-261/viewer?title_no=1571&episode_no=261 70 41.46968317

In [83]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[40:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

finished https://www.webtoons.com/en/romance/so-i-married-the-antifan/episode-5/viewer?title_no=2739&episode_no=6 1 1.4115989208221436
finished https://www.webtoons.com/en/romance/truebeauty/episode-3/viewer?title_no=1436&episode_no=4 2 3.132546901702881
finished https://www.webtoons.com/en/romance/to-love-your-enemy/episode-8/viewer?title_no=2114&episode_no=8 3 4.255337953567505
finished https://www.webtoons.com/en/action/i-get-stronger-the-more-i-eat/episode-3/viewer?title_no=3199&episode_no=4 4 6.114068031311035
finished https://www.webtoons.com/en/drama/mystic-prince/ep-6-some-blessings/viewer?title_no=4737&episode_no=6 5 6.557674169540405
finished https://www.webtoons.com/en/action/the-first-hunter/episode-0-prologue/viewer?title_no=3040&episode_no=1 6 7.01115083694458
finished https://www.webtoons.com/en/romance/nice-to-meet-you/episode-8/viewer?title_no=2066&episode_no=8 7 8.80309796333313
finished https://www.webtoons.com/en/thriller/shotgun-boy/ep-3/viewer?title_no=2534&episod

finished https://www.webtoons.com/en/action/hectopascal/s2-episode-55/viewer?title_no=5033&episode_no=55 64 71.43714594841003
finished https://www.webtoons.com/en/thriller/class-1-9/episode-47/viewer?title_no=4955&episode_no=47 65 71.92622494697571
finished https://www.webtoons.com/en/horror/webtoon-at-midnight/i-discovered-a-town-where-i-could-finally-be-happy-with-family-but-then-their-behavior-took-a-strange-turn-34/viewer?title_no=5773&episode_no=30 66 72.46538305282593
finished https://www.webtoons.com/en/action/mr-baek/ep-6-the-council-/viewer?title_no=5437&episode_no=6 67 72.93397784233093
finished https://www.webtoons.com/en/tiptoon/webtoon-now/october-return-week-2023/viewer?title_no=2025&episode_no=266 68 75.93696904182434
finished https://www.webtoons.com/en/fantasy/here-there-be-dragons/episode-52/viewer?title_no=2726&episode_no=52 69 78.90745782852173
finished https://www.webtoons.com/en/action/eleceed/episode-260/viewer?title_no=1571&episode_no=260 70 79.57911896705627
fi

ConnectError: [Errno 8] nodename nor servname provided, or not known

In [84]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[41:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

finished https://www.webtoons.com/en/romance/so-i-married-the-antifan/episode-4/viewer?title_no=2739&episode_no=5 1 0.7931098937988281
finished https://www.webtoons.com/en/romance/truebeauty/episode-2/viewer?title_no=1436&episode_no=3 2 1.257652997970581
finished https://www.webtoons.com/en/romance/to-love-your-enemy/episode-7/viewer?title_no=2114&episode_no=7 3 1.7398710250854492
finished https://www.webtoons.com/en/action/i-get-stronger-the-more-i-eat/episode-2/viewer?title_no=3199&episode_no=3 4 2.3129160404205322
finished https://www.webtoons.com/en/fantasy/the-king-and-the-paladin/episode-3/viewer?title_no=4735&episode_no=3 5 4.168072938919067
finished https://www.webtoons.com/en/fantasy/how-to-survive-a-romance-fantasy/episode-7/viewer?title_no=2855&episode_no=7 6 7.245759010314941
finished https://www.webtoons.com/en/romance/nice-to-meet-you/episode-7/viewer?title_no=2066&episode_no=7 7 10.184814929962158
finished https://www.webtoons.com/en/thriller/shotgun-boy/ep-2/viewer?titl

finished https://www.webtoons.com/en/action/hectopascal/s2-episode-54/viewer?title_no=5033&episode_no=54 64 68.4817419052124
finished https://www.webtoons.com/en/thriller/class-1-9/episode-46/viewer?title_no=4955&episode_no=46 65 69.21881794929504
finished https://www.webtoons.com/en/horror/webtoon-at-midnight/i-discovered-a-town-where-i-could-finally-be-happy-with-family-but-then-their-behavior-took-a-strange-turn-24/viewer?title_no=5773&episode_no=29 66 69.91780304908752
finished https://www.webtoons.com/en/action/mr-baek/ep-5-the-great-mountain/viewer?title_no=5437&episode_no=5 67 70.60680079460144
finished https://www.webtoons.com/en/tiptoon/webtoon-now/october-launch-week-2023-romance/viewer?title_no=2025&episode_no=265 68 73.94159007072449
finished https://www.webtoons.com/en/fantasy/here-there-be-dragons/episode-51/viewer?title_no=2726&episode_no=51 69 74.56748700141907
finished https://www.webtoons.com/en/action/eleceed/episode-259/viewer?title_no=1571&episode_no=259 70 75.7592

ConnectError: [Errno 8] nodename nor servname provided, or not known

In [83]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[42:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

finished https://www.webtoons.com/en/romance/so-i-married-the-antifan/episode-5/viewer?title_no=2739&episode_no=6 1 1.4115989208221436
finished https://www.webtoons.com/en/romance/truebeauty/episode-3/viewer?title_no=1436&episode_no=4 2 3.132546901702881
finished https://www.webtoons.com/en/romance/to-love-your-enemy/episode-8/viewer?title_no=2114&episode_no=8 3 4.255337953567505
finished https://www.webtoons.com/en/action/i-get-stronger-the-more-i-eat/episode-3/viewer?title_no=3199&episode_no=4 4 6.114068031311035
finished https://www.webtoons.com/en/drama/mystic-prince/ep-6-some-blessings/viewer?title_no=4737&episode_no=6 5 6.557674169540405
finished https://www.webtoons.com/en/action/the-first-hunter/episode-0-prologue/viewer?title_no=3040&episode_no=1 6 7.01115083694458
finished https://www.webtoons.com/en/romance/nice-to-meet-you/episode-8/viewer?title_no=2066&episode_no=8 7 8.80309796333313
finished https://www.webtoons.com/en/thriller/shotgun-boy/ep-3/viewer?title_no=2534&episod

finished https://www.webtoons.com/en/action/hectopascal/s2-episode-55/viewer?title_no=5033&episode_no=55 64 71.43714594841003
finished https://www.webtoons.com/en/thriller/class-1-9/episode-47/viewer?title_no=4955&episode_no=47 65 71.92622494697571
finished https://www.webtoons.com/en/horror/webtoon-at-midnight/i-discovered-a-town-where-i-could-finally-be-happy-with-family-but-then-their-behavior-took-a-strange-turn-34/viewer?title_no=5773&episode_no=30 66 72.46538305282593
finished https://www.webtoons.com/en/action/mr-baek/ep-6-the-council-/viewer?title_no=5437&episode_no=6 67 72.93397784233093
finished https://www.webtoons.com/en/tiptoon/webtoon-now/october-return-week-2023/viewer?title_no=2025&episode_no=266 68 75.93696904182434
finished https://www.webtoons.com/en/fantasy/here-there-be-dragons/episode-52/viewer?title_no=2726&episode_no=52 69 78.90745782852173
finished https://www.webtoons.com/en/action/eleceed/episode-260/viewer?title_no=1571&episode_no=260 70 79.57911896705627
fi

ConnectError: [Errno 8] nodename nor servname provided, or not known

In [83]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[43:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

finished https://www.webtoons.com/en/romance/so-i-married-the-antifan/episode-5/viewer?title_no=2739&episode_no=6 1 1.4115989208221436
finished https://www.webtoons.com/en/romance/truebeauty/episode-3/viewer?title_no=1436&episode_no=4 2 3.132546901702881
finished https://www.webtoons.com/en/romance/to-love-your-enemy/episode-8/viewer?title_no=2114&episode_no=8 3 4.255337953567505
finished https://www.webtoons.com/en/action/i-get-stronger-the-more-i-eat/episode-3/viewer?title_no=3199&episode_no=4 4 6.114068031311035
finished https://www.webtoons.com/en/drama/mystic-prince/ep-6-some-blessings/viewer?title_no=4737&episode_no=6 5 6.557674169540405
finished https://www.webtoons.com/en/action/the-first-hunter/episode-0-prologue/viewer?title_no=3040&episode_no=1 6 7.01115083694458
finished https://www.webtoons.com/en/romance/nice-to-meet-you/episode-8/viewer?title_no=2066&episode_no=8 7 8.80309796333313
finished https://www.webtoons.com/en/thriller/shotgun-boy/ep-3/viewer?title_no=2534&episod

finished https://www.webtoons.com/en/action/hectopascal/s2-episode-55/viewer?title_no=5033&episode_no=55 64 71.43714594841003
finished https://www.webtoons.com/en/thriller/class-1-9/episode-47/viewer?title_no=4955&episode_no=47 65 71.92622494697571
finished https://www.webtoons.com/en/horror/webtoon-at-midnight/i-discovered-a-town-where-i-could-finally-be-happy-with-family-but-then-their-behavior-took-a-strange-turn-34/viewer?title_no=5773&episode_no=30 66 72.46538305282593
finished https://www.webtoons.com/en/action/mr-baek/ep-6-the-council-/viewer?title_no=5437&episode_no=6 67 72.93397784233093
finished https://www.webtoons.com/en/tiptoon/webtoon-now/october-return-week-2023/viewer?title_no=2025&episode_no=266 68 75.93696904182434
finished https://www.webtoons.com/en/fantasy/here-there-be-dragons/episode-52/viewer?title_no=2726&episode_no=52 69 78.90745782852173
finished https://www.webtoons.com/en/action/eleceed/episode-260/viewer?title_no=1571&episode_no=260 70 79.57911896705627
fi

ConnectError: [Errno 8] nodename nor servname provided, or not known

In [83]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[44:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

finished https://www.webtoons.com/en/romance/so-i-married-the-antifan/episode-5/viewer?title_no=2739&episode_no=6 1 1.4115989208221436
finished https://www.webtoons.com/en/romance/truebeauty/episode-3/viewer?title_no=1436&episode_no=4 2 3.132546901702881
finished https://www.webtoons.com/en/romance/to-love-your-enemy/episode-8/viewer?title_no=2114&episode_no=8 3 4.255337953567505
finished https://www.webtoons.com/en/action/i-get-stronger-the-more-i-eat/episode-3/viewer?title_no=3199&episode_no=4 4 6.114068031311035
finished https://www.webtoons.com/en/drama/mystic-prince/ep-6-some-blessings/viewer?title_no=4737&episode_no=6 5 6.557674169540405
finished https://www.webtoons.com/en/action/the-first-hunter/episode-0-prologue/viewer?title_no=3040&episode_no=1 6 7.01115083694458
finished https://www.webtoons.com/en/romance/nice-to-meet-you/episode-8/viewer?title_no=2066&episode_no=8 7 8.80309796333313
finished https://www.webtoons.com/en/thriller/shotgun-boy/ep-3/viewer?title_no=2534&episod

finished https://www.webtoons.com/en/action/hectopascal/s2-episode-55/viewer?title_no=5033&episode_no=55 64 71.43714594841003
finished https://www.webtoons.com/en/thriller/class-1-9/episode-47/viewer?title_no=4955&episode_no=47 65 71.92622494697571
finished https://www.webtoons.com/en/horror/webtoon-at-midnight/i-discovered-a-town-where-i-could-finally-be-happy-with-family-but-then-their-behavior-took-a-strange-turn-34/viewer?title_no=5773&episode_no=30 66 72.46538305282593
finished https://www.webtoons.com/en/action/mr-baek/ep-6-the-council-/viewer?title_no=5437&episode_no=6 67 72.93397784233093
finished https://www.webtoons.com/en/tiptoon/webtoon-now/october-return-week-2023/viewer?title_no=2025&episode_no=266 68 75.93696904182434
finished https://www.webtoons.com/en/fantasy/here-there-be-dragons/episode-52/viewer?title_no=2726&episode_no=52 69 78.90745782852173
finished https://www.webtoons.com/en/action/eleceed/episode-260/viewer?title_no=1571&episode_no=260 70 79.57911896705627
fi

ConnectError: [Errno 8] nodename nor servname provided, or not known

In [83]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[45:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

finished https://www.webtoons.com/en/romance/so-i-married-the-antifan/episode-5/viewer?title_no=2739&episode_no=6 1 1.4115989208221436
finished https://www.webtoons.com/en/romance/truebeauty/episode-3/viewer?title_no=1436&episode_no=4 2 3.132546901702881
finished https://www.webtoons.com/en/romance/to-love-your-enemy/episode-8/viewer?title_no=2114&episode_no=8 3 4.255337953567505
finished https://www.webtoons.com/en/action/i-get-stronger-the-more-i-eat/episode-3/viewer?title_no=3199&episode_no=4 4 6.114068031311035
finished https://www.webtoons.com/en/drama/mystic-prince/ep-6-some-blessings/viewer?title_no=4737&episode_no=6 5 6.557674169540405
finished https://www.webtoons.com/en/action/the-first-hunter/episode-0-prologue/viewer?title_no=3040&episode_no=1 6 7.01115083694458
finished https://www.webtoons.com/en/romance/nice-to-meet-you/episode-8/viewer?title_no=2066&episode_no=8 7 8.80309796333313
finished https://www.webtoons.com/en/thriller/shotgun-boy/ep-3/viewer?title_no=2534&episod

finished https://www.webtoons.com/en/action/hectopascal/s2-episode-55/viewer?title_no=5033&episode_no=55 64 71.43714594841003
finished https://www.webtoons.com/en/thriller/class-1-9/episode-47/viewer?title_no=4955&episode_no=47 65 71.92622494697571
finished https://www.webtoons.com/en/horror/webtoon-at-midnight/i-discovered-a-town-where-i-could-finally-be-happy-with-family-but-then-their-behavior-took-a-strange-turn-34/viewer?title_no=5773&episode_no=30 66 72.46538305282593
finished https://www.webtoons.com/en/action/mr-baek/ep-6-the-council-/viewer?title_no=5437&episode_no=6 67 72.93397784233093
finished https://www.webtoons.com/en/tiptoon/webtoon-now/october-return-week-2023/viewer?title_no=2025&episode_no=266 68 75.93696904182434
finished https://www.webtoons.com/en/fantasy/here-there-be-dragons/episode-52/viewer?title_no=2726&episode_no=52 69 78.90745782852173
finished https://www.webtoons.com/en/action/eleceed/episode-260/viewer?title_no=1571&episode_no=260 70 79.57911896705627
fi

ConnectError: [Errno 8] nodename nor servname provided, or not known

In [83]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[46:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

finished https://www.webtoons.com/en/romance/so-i-married-the-antifan/episode-5/viewer?title_no=2739&episode_no=6 1 1.4115989208221436
finished https://www.webtoons.com/en/romance/truebeauty/episode-3/viewer?title_no=1436&episode_no=4 2 3.132546901702881
finished https://www.webtoons.com/en/romance/to-love-your-enemy/episode-8/viewer?title_no=2114&episode_no=8 3 4.255337953567505
finished https://www.webtoons.com/en/action/i-get-stronger-the-more-i-eat/episode-3/viewer?title_no=3199&episode_no=4 4 6.114068031311035
finished https://www.webtoons.com/en/drama/mystic-prince/ep-6-some-blessings/viewer?title_no=4737&episode_no=6 5 6.557674169540405
finished https://www.webtoons.com/en/action/the-first-hunter/episode-0-prologue/viewer?title_no=3040&episode_no=1 6 7.01115083694458
finished https://www.webtoons.com/en/romance/nice-to-meet-you/episode-8/viewer?title_no=2066&episode_no=8 7 8.80309796333313
finished https://www.webtoons.com/en/thriller/shotgun-boy/ep-3/viewer?title_no=2534&episod

finished https://www.webtoons.com/en/action/hectopascal/s2-episode-55/viewer?title_no=5033&episode_no=55 64 71.43714594841003
finished https://www.webtoons.com/en/thriller/class-1-9/episode-47/viewer?title_no=4955&episode_no=47 65 71.92622494697571
finished https://www.webtoons.com/en/horror/webtoon-at-midnight/i-discovered-a-town-where-i-could-finally-be-happy-with-family-but-then-their-behavior-took-a-strange-turn-34/viewer?title_no=5773&episode_no=30 66 72.46538305282593
finished https://www.webtoons.com/en/action/mr-baek/ep-6-the-council-/viewer?title_no=5437&episode_no=6 67 72.93397784233093
finished https://www.webtoons.com/en/tiptoon/webtoon-now/october-return-week-2023/viewer?title_no=2025&episode_no=266 68 75.93696904182434
finished https://www.webtoons.com/en/fantasy/here-there-be-dragons/episode-52/viewer?title_no=2726&episode_no=52 69 78.90745782852173
finished https://www.webtoons.com/en/action/eleceed/episode-260/viewer?title_no=1571&episode_no=260 70 79.57911896705627
fi

ConnectError: [Errno 8] nodename nor servname provided, or not known

In [83]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[47:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

finished https://www.webtoons.com/en/romance/so-i-married-the-antifan/episode-5/viewer?title_no=2739&episode_no=6 1 1.4115989208221436
finished https://www.webtoons.com/en/romance/truebeauty/episode-3/viewer?title_no=1436&episode_no=4 2 3.132546901702881
finished https://www.webtoons.com/en/romance/to-love-your-enemy/episode-8/viewer?title_no=2114&episode_no=8 3 4.255337953567505
finished https://www.webtoons.com/en/action/i-get-stronger-the-more-i-eat/episode-3/viewer?title_no=3199&episode_no=4 4 6.114068031311035
finished https://www.webtoons.com/en/drama/mystic-prince/ep-6-some-blessings/viewer?title_no=4737&episode_no=6 5 6.557674169540405
finished https://www.webtoons.com/en/action/the-first-hunter/episode-0-prologue/viewer?title_no=3040&episode_no=1 6 7.01115083694458
finished https://www.webtoons.com/en/romance/nice-to-meet-you/episode-8/viewer?title_no=2066&episode_no=8 7 8.80309796333313
finished https://www.webtoons.com/en/thriller/shotgun-boy/ep-3/viewer?title_no=2534&episod

finished https://www.webtoons.com/en/action/hectopascal/s2-episode-55/viewer?title_no=5033&episode_no=55 64 71.43714594841003
finished https://www.webtoons.com/en/thriller/class-1-9/episode-47/viewer?title_no=4955&episode_no=47 65 71.92622494697571
finished https://www.webtoons.com/en/horror/webtoon-at-midnight/i-discovered-a-town-where-i-could-finally-be-happy-with-family-but-then-their-behavior-took-a-strange-turn-34/viewer?title_no=5773&episode_no=30 66 72.46538305282593
finished https://www.webtoons.com/en/action/mr-baek/ep-6-the-council-/viewer?title_no=5437&episode_no=6 67 72.93397784233093
finished https://www.webtoons.com/en/tiptoon/webtoon-now/october-return-week-2023/viewer?title_no=2025&episode_no=266 68 75.93696904182434
finished https://www.webtoons.com/en/fantasy/here-there-be-dragons/episode-52/viewer?title_no=2726&episode_no=52 69 78.90745782852173
finished https://www.webtoons.com/en/action/eleceed/episode-260/viewer?title_no=1571&episode_no=260 70 79.57911896705627
fi

ConnectError: [Errno 8] nodename nor servname provided, or not known

In [83]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[48:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

finished https://www.webtoons.com/en/romance/so-i-married-the-antifan/episode-5/viewer?title_no=2739&episode_no=6 1 1.4115989208221436
finished https://www.webtoons.com/en/romance/truebeauty/episode-3/viewer?title_no=1436&episode_no=4 2 3.132546901702881
finished https://www.webtoons.com/en/romance/to-love-your-enemy/episode-8/viewer?title_no=2114&episode_no=8 3 4.255337953567505
finished https://www.webtoons.com/en/action/i-get-stronger-the-more-i-eat/episode-3/viewer?title_no=3199&episode_no=4 4 6.114068031311035
finished https://www.webtoons.com/en/drama/mystic-prince/ep-6-some-blessings/viewer?title_no=4737&episode_no=6 5 6.557674169540405
finished https://www.webtoons.com/en/action/the-first-hunter/episode-0-prologue/viewer?title_no=3040&episode_no=1 6 7.01115083694458
finished https://www.webtoons.com/en/romance/nice-to-meet-you/episode-8/viewer?title_no=2066&episode_no=8 7 8.80309796333313
finished https://www.webtoons.com/en/thriller/shotgun-boy/ep-3/viewer?title_no=2534&episod

finished https://www.webtoons.com/en/action/hectopascal/s2-episode-55/viewer?title_no=5033&episode_no=55 64 71.43714594841003
finished https://www.webtoons.com/en/thriller/class-1-9/episode-47/viewer?title_no=4955&episode_no=47 65 71.92622494697571
finished https://www.webtoons.com/en/horror/webtoon-at-midnight/i-discovered-a-town-where-i-could-finally-be-happy-with-family-but-then-their-behavior-took-a-strange-turn-34/viewer?title_no=5773&episode_no=30 66 72.46538305282593
finished https://www.webtoons.com/en/action/mr-baek/ep-6-the-council-/viewer?title_no=5437&episode_no=6 67 72.93397784233093
finished https://www.webtoons.com/en/tiptoon/webtoon-now/october-return-week-2023/viewer?title_no=2025&episode_no=266 68 75.93696904182434
finished https://www.webtoons.com/en/fantasy/here-there-be-dragons/episode-52/viewer?title_no=2726&episode_no=52 69 78.90745782852173
finished https://www.webtoons.com/en/action/eleceed/episode-260/viewer?title_no=1571&episode_no=260 70 79.57911896705627
fi

ConnectError: [Errno 8] nodename nor servname provided, or not known

In [83]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[49:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

finished https://www.webtoons.com/en/romance/so-i-married-the-antifan/episode-5/viewer?title_no=2739&episode_no=6 1 1.4115989208221436
finished https://www.webtoons.com/en/romance/truebeauty/episode-3/viewer?title_no=1436&episode_no=4 2 3.132546901702881
finished https://www.webtoons.com/en/romance/to-love-your-enemy/episode-8/viewer?title_no=2114&episode_no=8 3 4.255337953567505
finished https://www.webtoons.com/en/action/i-get-stronger-the-more-i-eat/episode-3/viewer?title_no=3199&episode_no=4 4 6.114068031311035
finished https://www.webtoons.com/en/drama/mystic-prince/ep-6-some-blessings/viewer?title_no=4737&episode_no=6 5 6.557674169540405
finished https://www.webtoons.com/en/action/the-first-hunter/episode-0-prologue/viewer?title_no=3040&episode_no=1 6 7.01115083694458
finished https://www.webtoons.com/en/romance/nice-to-meet-you/episode-8/viewer?title_no=2066&episode_no=8 7 8.80309796333313
finished https://www.webtoons.com/en/thriller/shotgun-boy/ep-3/viewer?title_no=2534&episod

finished https://www.webtoons.com/en/action/hectopascal/s2-episode-55/viewer?title_no=5033&episode_no=55 64 71.43714594841003
finished https://www.webtoons.com/en/thriller/class-1-9/episode-47/viewer?title_no=4955&episode_no=47 65 71.92622494697571
finished https://www.webtoons.com/en/horror/webtoon-at-midnight/i-discovered-a-town-where-i-could-finally-be-happy-with-family-but-then-their-behavior-took-a-strange-turn-34/viewer?title_no=5773&episode_no=30 66 72.46538305282593
finished https://www.webtoons.com/en/action/mr-baek/ep-6-the-council-/viewer?title_no=5437&episode_no=6 67 72.93397784233093
finished https://www.webtoons.com/en/tiptoon/webtoon-now/october-return-week-2023/viewer?title_no=2025&episode_no=266 68 75.93696904182434
finished https://www.webtoons.com/en/fantasy/here-there-be-dragons/episode-52/viewer?title_no=2726&episode_no=52 69 78.90745782852173
finished https://www.webtoons.com/en/action/eleceed/episode-260/viewer?title_no=1571&episode_no=260 70 79.57911896705627
fi

ConnectError: [Errno 8] nodename nor servname provided, or not known

In [ ]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[50:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

In [ ]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[51:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

In [ ]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[52:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

In [ ]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[53:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

In [ ]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[54:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

In [ ]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[55:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

In [ ]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[56:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

In [ ]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[57:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

In [ ]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[58:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

In [ ]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[59:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())

In [ ]:
import random
async def download_image(url, directory, filename, semaphore):
    querystring = {"type": "q90"}
    headers = {"referer": "https://www.webtoons.com/"}

    async with httpx.AsyncClient() as client:
        async with semaphore:
            response = await client.get(url, headers=headers)
            if response.status_code == 200:
                async with aiofiles.open(os.path.join(directory, filename), "wb") as f:
                    await f.write(response.content)

async def main():
    start_time = time.time()
    image_directory = 'images'
    if not os.path.exists(image_directory):
        os.makedirs(image_directory)
    # Download images for each episode URL asynchronously
    tasks = []
    counter = 0
    for episode_url, genre in list(episode_url_genre_dict.items())[60:len(episode_url_genre_dict):90]:
        genre_directory = f"{image_directory}/{genre}"
        if not os.path.exists(genre_directory):
            os.makedirs(genre_directory)
        async with httpx.AsyncClient() as client:
            r = await client.get(episode_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        for img in list(soup.find_all('img', class_='_images'))[0:100:8]:
            img_url = re.sub("\?type=\w*", "", img['data-url'])
            filename = img_url.split('/')[-1]
            tasks.append(download_image(img_url, genre_directory, filename, semaphore))  # Pass semaphore here
        counter += 1
        print(f"finished {episode_url} {counter} {time.time() - start_time}")
    print(time.time() - start_time)
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())